In [1]:
import pandas as pd 
import numpy as np 

# DATA SOURCE

##  STATE

- Read Data from Source 1 (State, Capital, StateID)

In [2]:
state=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states.json',dtype=str)
state=state[state['name']!='LUAR NEGARA']
state

,name,name_long,id,code2,code3,capital
0,JOHOR,JOHOR DARUL TAKZIM,1,JH,JOH,JOHOR BAHRU
1,KEDAH,KEDAH DARUL AMAN,2,KD,KED,ALOR SETAR
2,KELANTAN,KELANTAN DARUL NAIM,3,KT,KEL,KOTA BAHRU
3,MELAKA,MELAKA BANDARAYA BERSEJARAH,4,ML,MLK,BANDAR MELAKA
4,NEGERI SEMBILAN,NEGERI SEMBILAN DARUL KHUSUS,5,NS,NSN,SEREMBAN
5,PAHANG,PAHANG DARUL MAKMUR,6,PH,PAH,KUANTAN
6,PULAU PINANG,PULAU PINANG PULAU MUTIARA,7,PN,PEN,GEORGE TOWN
7,PERAK,PERAK DARUL RIDZUAN,8,PR,PRK,IPOH
8,PERLIS,PERLIS INDERA KAYANGAN,9,PL,PER,KANGAR
9,SELANGOR,SELANGOR DARUL EHSAN,10,SG,SEL,SHAH ALAM


##  DISTRICT

- Read Data from Source 2 (StateID, District, DistrictID)

In [3]:
district=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states/district.json',dtype=str)
district=district[district['state_id']!='nan'].drop(columns='//comment')
district

,id,state_id,name,code
1,900,9,PERLIS,PER
2,201,2,KOTA SETAR,KSE
3,202,2,KUBANG PASU,KPA
4,203,2,PADANG TERAP,PTE
5,204,2,LANGKAWI,LAN
...,...,...,...,...
122,1307,13,KAPIT,KAP
123,1308,13,SAMARAHAN,SAM
124,1309,13,BINTULU,BIN
125,1310,13,MUKAH,MUK


##  SUBDISTRICT

- Read Data from Source 3 (SubDistrict, SubDistrictID, DistrictID)

In [4]:
mukim=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states/mukim.json',dtype=str)
mukim=mukim[mukim['name']!='-']#.drop(columns='//comment')
mukim

,id,name,district_id
0,090001,ABI,900
1,090002,ARAU,900
2,090003,BERSERI,900
3,090004,CHUPING,900
4,090005,UTAN AJI,900
...,...,...,...
1985,130947,ANAP,1309
1986,130948,KAKUS,1309
1987,130954,JELALONG,1309
1988,130955,BINIA,1309


In [5]:
mukim[mukim['name'].isnull()]

,id,name,district_id


### Transformation

- Left Join (mukim,district) on (DistrictID,ID)

In [6]:
mukim_district=\
pd.merge(
    mukim.drop(columns='id').rename(columns={'name': 'mukim'}),
    district.drop(columns='code').rename(columns={'name': 'district'}),
    how='left',
    left_on='district_id',
    right_on='id'
)

mukim_district

,mukim,district_id,id,state_id,district
0,ABI,900,900,9,PERLIS
1,ARAU,900,900,9,PERLIS
2,BERSERI,900,900,9,PERLIS
3,CHUPING,900,900,9,PERLIS
4,UTAN AJI,900,900,9,PERLIS
...,...,...,...,...,...
1985,ANAP,1309,1309,13,BINTULU
1986,KAKUS,1309,1309,13,BINTULU
1987,JELALONG,1309,1309,13,BINTULU
1988,BINIA,1309,1309,13,BINTULU


In [7]:
mukim_district[mukim_district['mukim'].isnull()]

,mukim,district_id,id,state_id,district


- Left Join (mukim_district, state) on (StateID, ID)

In [8]:
mukim_district_state=pd.merge(mukim_district.drop(columns=['district_id','id']),state,how='left',left_on='state_id',right_on='id').drop(columns=['name_long','capital','id','code2','code3']).rename(columns={'name': 'state'})
mukim_district_state

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


In [9]:
mukim_district_state[mukim_district_state['mukim'].isnull()]

,mukim,state_id,district,state


### Imputation

- Identify Mukim with Null State

In [10]:
mukim_district_state[mukim_district_state['state'].isnull()]

,mukim,state_id,district,state
1294,AMPANG,NaN,NaN,NaN
1295,BATU,NaN,NaN,NaN
1296,CHERAS,NaN,NaN,NaN
1297,ULU KELANG,NaN,NaN,NaN
1298,KUALA LUMPUR,NaN,NaN,NaN
1299,PETALING,NaN,NaN,NaN
1300,SETAPAK,NaN,NaN,NaN
1301,BANDAR KUALA LUMPUR,NaN,NaN,NaN
1302,BANDAR PETALING JAYA,NaN,NaN,NaN
1303,BANDAR BANDAR BAHARU SUNGAI BESI,NaN,NaN,NaN


In [11]:
mukim_district_state[mukim_district_state['mukim'].isnull()]

,mukim,state_id,district,state


- Fill in Null with Logical Value

In [12]:
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='AMPANG','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='BATU','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='CHERAS','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='ULU KELANG','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='KUALA LUMPUR','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PETALING','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='SETAPAK','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='BANDAR KUALA LUMPUR','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='BANDAR PETALING JAYA','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='BANDAR BANDAR BAHARU SUNGAI BESI','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN BATU','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN BATU CAVES','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN KEPONG','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN KUALA PAUH','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN PETALING','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN SALAK SOUTH','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN SUNGAI PENCHALA','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='LABUAN','WP LABUAN',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='DONGGONGON','SABAH',mukim_district_state['state'])


mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='AMPANG','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='BATU','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='CHERAS','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='ULU KELANG','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='KUALA LUMPUR','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='PETALING','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='SETAPAK','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='BANDAR KUALA LUMPUR','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='BANDAR PETALING JAYA','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='BANDAR BANDAR BAHARU SUNGAI BESI','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='PEKAN BATU','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='PEKAN BATU CAVES','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='PEKAN KEPONG','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='PEKAN KUALA PAUH','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='PEKAN PETALING','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='PEKAN SALAK SOUTH','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='PEKAN SUNGAI PENCHALA','14',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='LABUAN','15',mukim_district_state['state_id'])
mukim_district_state['state_id']=np.where(mukim_district_state['mukim']=='DONGGONGON','12',mukim_district_state['state_id'])

mukim_district_state


,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


In [13]:
mukim_district_state[mukim_district_state['mukim'].isnull()]

,mukim,state_id,district,state


In [18]:
mukim_district_state.drop_duplicates(subset=['mukim','state_id','district','state'], keep='first', inplace=True)
mukim_district_state

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


In [ ]:
#mukim_district_state['address_partial']=mukim_district_state['mukim']+','+mukim_district_state['district']+','+mukim_district_state['state']
#mukim_district_state

In [19]:
set(mukim_district_state['state'].to_list())

{'JOHOR',
 'KEDAH',
 'KELANTAN',
 'MELAKA',
 'NEGERI SEMBILAN',
 'PAHANG',
 'PERAK',
 'PERLIS',
 'PULAU PINANG',
 'SABAH',
 'SARAWAK',
 'SELANGOR',
 'TERENGGANU',
 'WP KUALA LUMPUR',
 'WP LABUAN',
 'WP PUTRAJAYA'}

## LOCALITY

- Read Data from Source 4 (Location, PostCode, District, State)

In [20]:
location=pd.read_csv('../data_source/Malaysia_Postcode-postcodes.csv',dtype=str,header=None)
location.columns=['postcode','location','district','state']
location

,postcode,location,district,state
0,01000,Alor Redeh,Kangar,PLS
1,01000,Alor Tok Terang,Kangar,PLS
2,01000,Bohor Gelam,Kangar,PLS
3,01000,Jalan Abi Tok Hashim,Kangar,PLS
4,01000,Jalan Alor Sena,Kangar,PLS
...,...,...,...,...
56229,98850,Sungai Seloating,Lawas,SRW
56230,98850,Trusan,Lawas,SRW
56231,98850,Tuaran,Lawas,SRW
56232,98857,Peti Surat,Lawas,SRW


- Check if postcode in location data is missing

In [21]:
location.isnull().sum()

postcode    0
location    0
district    0
state       0
dtype: int64

### Transformation

- DataFrame (location) Transformation to Standardize State as of (mukim_district_state)

In [22]:
location['state']=np.where(location['state']=='JHR','JOHOR',location['state'])
location['state']=np.where(location['state']=='KDH','KEDAH',location['state'])
location['state']=np.where(location['state']=='KTN', 'KELANTAN', location['state'])
location['state']=np.where(location['state']=='KUL', 'WP KUALA LUMPUR', location['state'])
location['state']=np.where(location['state']=='LBN', 'WP LABUAN', location['state'])
location['state']=np.where(location['state']=='MLK', 'MELAKA', location['state'])
location['state']=np.where(location['state']=='NSN', 'NEGERI SEMBILAN', location['state'])
location['state']=np.where(location['state']=='PHG', 'PAHANG', location['state'])
location['state']=np.where(location['state']=='PJY', 'WP PUTRAJAYA', location['state'])
location['state']=np.where(location['state']=='PLS', 'PERLIS', location['state'])
location['state']=np.where(location['state']=='PNG', 'PULAU PINANG', location['state'])
location['state']=np.where(location['state']=='PRK', 'PERAK', location['state'])
location['state']=np.where(location['state']=='SBH', 'SABAH', location['state'])
location['state']=np.where(location['state']=='SGR', 'SELANGOR', location['state'])
location['state']=np.where(location['state']=='SRW', 'SARAWAK', location['state'])
location['state']=np.where(location['state']=='TRG', 'TERENGGANU', location['state'])

location

,postcode,location,district,state
0,01000,Alor Redeh,Kangar,PERLIS
1,01000,Alor Tok Terang,Kangar,PERLIS
2,01000,Bohor Gelam,Kangar,PERLIS
3,01000,Jalan Abi Tok Hashim,Kangar,PERLIS
4,01000,Jalan Alor Sena,Kangar,PERLIS
...,...,...,...,...
56229,98850,Sungai Seloating,Lawas,SARAWAK
56230,98850,Trusan,Lawas,SARAWAK
56231,98850,Tuaran,Lawas,SARAWAK
56232,98857,Peti Surat,Lawas,SARAWAK


In [23]:
location['district']=location['district'].str.upper()
location['location']=location['location'].str.upper()
location

,postcode,location,district,state
0,01000,ALOR REDEH,KANGAR,PERLIS
1,01000,ALOR TOK TERANG,KANGAR,PERLIS
2,01000,BOHOR GELAM,KANGAR,PERLIS
3,01000,JALAN ABI TOK HASHIM,KANGAR,PERLIS
4,01000,JALAN ALOR SENA,KANGAR,PERLIS
...,...,...,...,...
56229,98850,SUNGAI SELOATING,LAWAS,SARAWAK
56230,98850,TRUSAN,LAWAS,SARAWAK
56231,98850,TUARAN,LAWAS,SARAWAK
56232,98857,PETI SURAT,LAWAS,SARAWAK


In [24]:
mukim_district_state['state']=mukim_district_state['state'].str.upper()
mukim_district_state['district']=mukim_district_state['district'].str.upper()
mukim_district_state['mukim']=mukim_district_state['mukim'].str.upper()
mukim_district_state

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


- Function to Return Postcode Value of Reference Table (location) if String Combination of Main Table (mukim_district_state) Exist 

In [25]:
def find_state_district_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['state'].lower() in text.lower()) and (row['district'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (state, district, location) for that particular postcode

def find_state_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['state'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (state, location) for that particular postcode

def find_district_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['district'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (district, location) for that particular postcode
 

- Test the functions

In [26]:
mukim_district_state_test = mukim_district_state.head(10)
mukim_district_state_test

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
5,JEJAWI,9,PERLIS,PERLIS
6,KAYANG,9,PERLIS,PERLIS
7,KECHOR,9,PERLIS,PERLIS
8,KUALA PERLIS,9,PERLIS,PERLIS
9,KURONG ANAI,9,PERLIS,PERLIS


In [27]:
'''
# test functions workability 
mukim_district_state_test['postcode1']=np.nan
mukim_district_state_test['postcode2']=np.nan
mukim_district_state_test['postcode3']=np.nan

mukim_district_state_test['postcode1'] = (mukim_district_state_test['mukim'].astype(str)+','+mukim_district_state_test['district'].astype(str)+','+mukim_district_state_test['state'].astype(str)).apply(lambda x: find_state_district_location(x, location)).to_list()
mukim_district_state_test['postcode2'] = (mukim_district_state_test['mukim'].astype(str)+','+mukim_district_state_test['district'].astype(str)+','+mukim_district_state_test['state'].astype(str)).apply(lambda x: find_state_location(x, location)).to_list()
mukim_district_state_test['postcode3'] = (mukim_district_state_test['mukim'].astype(str)+','+mukim_district_state_test['district'].astype(str)+','+mukim_district_state_test['state'].astype(str)).apply(lambda x: find_district_location(x, location)).to_list()
'''

# If mukim_district_state_test is a slice, make a copy first
mukim_district_state_test = mukim_district_state_test.copy()

# Assign new columns directly (no need for .to_list())
mukim_district_state_test['postcode1'] = (
    mukim_district_state_test['mukim'].astype(str) + ',' +
    mukim_district_state_test['district'].astype(str) + ',' +
    mukim_district_state_test['state'].astype(str)
).apply(lambda x: find_state_district_location(x, location))

mukim_district_state_test['postcode2'] = (
    mukim_district_state_test['mukim'].astype(str) + ',' +
    mukim_district_state_test['district'].astype(str) + ',' +
    mukim_district_state_test['state'].astype(str)
).apply(lambda x: find_state_location(x, location))

mukim_district_state_test['postcode3'] = (
    mukim_district_state_test['mukim'].astype(str) + ',' +
    mukim_district_state_test['district'].astype(str) + ',' +
    mukim_district_state_test['state'].astype(str)
).apply(lambda x: find_district_location(x, location))

mukim_district_state_test

,mukim,state_id,district,state,postcode1,postcode2,postcode3
0,ABI,9,PERLIS,PERLIS,{},{},{}
1,ARAU,9,PERLIS,PERLIS,{02600},{02600},{02600}
2,BERSERI,9,PERLIS,PERLIS,{},{},{}
3,CHUPING,9,PERLIS,PERLIS,{},{},{}
4,UTAN AJI,9,PERLIS,PERLIS,{},{},{}
5,JEJAWI,9,PERLIS,PERLIS,{},{01000},{}
6,KAYANG,9,PERLIS,PERLIS,{},{},{06900}
7,KECHOR,9,PERLIS,PERLIS,{},{},{}
8,KUALA PERLIS,9,PERLIS,PERLIS,{02000},{02000},{02000}
9,KURONG ANAI,9,PERLIS,PERLIS,{},{},{}


In [ ]:
# apply functions 

mukim_district_state['postcode1']=np.nan
mukim_district_state['postcode2']=np.nan
mukim_district_state['postcode3']=np.nan

mukim_district_state['postcode1'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_state_district_location(x, location)).to_list()
mukim_district_state['postcode2'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_state_location(x, location)).to_list()
mukim_district_state['postcode3'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_district_location(x, location)).to_list()


- Split Postcode based on Combination of Scenario 1,2 and 3

In [ ]:
mukim_district_state['postcode1'] = mukim_district_state['postcode1'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state['postcode2'] = mukim_district_state['postcode2'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state['postcode3'] = mukim_district_state['postcode3'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state

In [ ]:
#mukim_district_state.to_csv('./data_preprocessed/prep1.csv',index=False)

In [28]:
mukim_district_state=pd.read_csv('../data_preprocessed/prep0.csv',dtype=str)
mukim_district_state
#mukim_district_state

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3
0,NaN,ALOR REDEH,KANGAR,PERLIS,NaN,NaN,NaN,NaN
1,NaN,ALOR TOK TERANG,KANGAR,PERLIS,NaN,NaN,NaN,NaN
2,NaN,BOHOR GELAM,KANGAR,PERLIS,NaN,NaN,NaN,NaN
3,NaN,JALAN ABI TOK HASHIM,KANGAR,PERLIS,NaN,NaN,NaN,NaN
4,NaN,JALAN ALOR SENA,KANGAR,PERLIS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
58219,97000,NaN,BINTULU,SARAWAK,ANAP,97000,97000,97000
58220,97000,NaN,BINTULU,SARAWAK,KAKUS,97000,"97000, 97200",97000
58221,97000,NaN,BINTULU,SARAWAK,JELALONG,97000,97000,97000
58222,97000,NaN,BINTULU,SARAWAK,BINIA,97000,97000,97000


In [30]:
mukim_district_state=mukim_district_state[mukim_district_state['mukim'].notnull()]
mukim_district_state

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3
56234,NaN,NaN,PERLIS,PERLIS,ABI,NaN,NaN,NaN
56235,02600,NaN,PERLIS,PERLIS,ARAU,02600,02600,02600
56236,NaN,NaN,PERLIS,PERLIS,BERSERI,NaN,NaN,NaN
56237,NaN,NaN,PERLIS,PERLIS,CHUPING,NaN,NaN,NaN
56238,NaN,NaN,PERLIS,PERLIS,UTAN AJI,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
58219,97000,NaN,BINTULU,SARAWAK,ANAP,97000,97000,97000
58220,97000,NaN,BINTULU,SARAWAK,KAKUS,97000,"97000, 97200",97000
58221,97000,NaN,BINTULU,SARAWAK,JELALONG,97000,97000,97000
58222,97000,NaN,BINTULU,SARAWAK,BINIA,97000,97000,97000


- Extract Postcode Value by Priority (1,2,3)

In [31]:
mukim_district_state['postcode']=mukim_district_state['postcode1']
mukim_district_state['postcode']=np.where(mukim_district_state['postcode'].isnull(),mukim_district_state['postcode2'],mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where(mukim_district_state['postcode'].isnull(),mukim_district_state['postcode3'],mukim_district_state['postcode'])
mukim_district_state

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\960387730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=mukim_district_state['postcode1']
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\960387730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where(mukim_district_state['postcode'].isnull(),mukim_district_state['postcode2'],mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\960387730.py:3: Set

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3
56234,NaN,NaN,PERLIS,PERLIS,ABI,NaN,NaN,NaN
56235,02600,NaN,PERLIS,PERLIS,ARAU,02600,02600,02600
56236,NaN,NaN,PERLIS,PERLIS,BERSERI,NaN,NaN,NaN
56237,NaN,NaN,PERLIS,PERLIS,CHUPING,NaN,NaN,NaN
56238,NaN,NaN,PERLIS,PERLIS,UTAN AJI,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
58219,97000,NaN,BINTULU,SARAWAK,ANAP,97000,97000,97000
58220,97000,NaN,BINTULU,SARAWAK,KAKUS,97000,"97000, 97200",97000
58221,97000,NaN,BINTULU,SARAWAK,JELALONG,97000,97000,97000
58222,97000,NaN,BINTULU,SARAWAK,BINIA,97000,97000,97000


In [34]:
mukim_district_state.isnull().sum()

postcode      357
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

- DataFrame (mukim_district_state) Transformation to Fill In Value for Null Postcode

In [38]:
# perlis
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ABI')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ARAU')&(mukim_district_state['state']=='PERLIS'), '02600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ARAU')&(mukim_district_state['state']=='PERLIS'), '02600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KANGAR')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERSERI')&(mukim_district_state['state']=='PERLIS'), '02400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHUPING')&(mukim_district_state['state']=='PERLIS'), '02500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='UTAN AJI')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JEJAWI')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAYANG')&(mukim_district_state['state']=='PERLIS'), '02000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KECHOR')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KURONG ANAI')&(mukim_district_state['state']=='PERLIS'), '02600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KURONG BATANG')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA PERLIS')&(mukim_district_state['state']=='PERLIS'), '02000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NGULANG')&(mukim_district_state['state']=='PERLIS'), '02400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ORAN')&(mukim_district_state['state']=='PERLIS'), '02500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG SIDING')&(mukim_district_state['state']=='PERLIS'), '02600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG PAUH')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PAYA')&(mukim_district_state['state']=='PERLIS'), '02400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA PERLIS')&(mukim_district_state['state']=='PERLIS'), '02000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAKI BUKIT')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SANGLANG')&(mukim_district_state['state']=='PERLIS'), '02800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SENA')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERIAB')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI ADAM')&(mukim_district_state['state']=='PERLIS'), '02600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TITI TINGGI')&(mukim_district_state['state']=='PERLIS'), '02100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='WANG BINTONG')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERLIS')&(mukim_district_state['mukim'].notnull())]
mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\3180167969.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ABI')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\3180167969.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode']

postcode      271
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [39]:
# penang
# show all row
# pd.set_option('display.max_rows', None)

# mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PULAU PINANG')]

# spt
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 2')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 3')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 4')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 5')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 6')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 7')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 8')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 9')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 10')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 11')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 12')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 13')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 14')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 15')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 16')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 17')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 18')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 19')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 20')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 21')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PRAI')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BUKIT MERTAJAM')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])

# spu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13110', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 2')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 3')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 4')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 5')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 6')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 7')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 8')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 9')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 10')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 11')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 12')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 13')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 14')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '12300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 15')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '12100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 16')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BUTTERWORTH')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '12000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KEPALA BATAS')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])

# sps
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14120', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 2')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 3')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 4')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 5')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 6')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 7')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 8')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 9')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 10')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14320', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 11')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 12')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 13')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14110', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 14')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 15')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14120', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 16')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR NIBONG TEBAL')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI BAKAP')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
# tl
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 13')&(mukim_district_state['district']=='TIMOR LAUT'), '11500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 14')&(mukim_district_state['district']=='TIMOR LAUT'), '11060', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 16')&(mukim_district_state['district']=='TIMOR LAUT'), '11500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 17')&(mukim_district_state['district']=='TIMOR LAUT'), '11100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 18')&(mukim_district_state['district']=='TIMOR LAUT'), '11200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU FERRINGGI')&(mukim_district_state['district']=='TIMOR LAUT'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GLUGOR')&(mukim_district_state['district']=='TIMOR LAUT'), '10150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GEORGE TOWN')&(mukim_district_state['district']=='TIMOR LAUT'), '10450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG BUNGAH')&(mukim_district_state['district']=='TIMOR LAUT'), '11200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR AYER ITAM')&(mukim_district_state['district']=='TIMOR LAUT'), '10460', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BUKIT BENDERA')&(mukim_district_state['district']=='TIMOR LAUT'), '11300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JELUTONG')&(mukim_district_state['district']=='TIMOR LAUT'), '11600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG TOKONG')&(mukim_district_state['district']=='TIMOR LAUT'), '10470', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG PINANG')&(mukim_district_state['district']=='TIMOR LAUT'), '10470', mukim_district_state['postcode'])
# bd
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='BARAT DAYA'), '11010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 2')&(mukim_district_state['district']=='BARAT DAYA'), '11050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 3')&(mukim_district_state['district']=='BARAT DAYA'), '11010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 4')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 5')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 6')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 7')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 8')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 9')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 10')&(mukim_district_state['district']=='BARAT DAYA'), '11900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 11')&(mukim_district_state['district']=='BARAT DAYA'), '11920', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 12')&(mukim_district_state['district']=='BARAT DAYA'), '11900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM A')&(mukim_district_state['district']=='BARAT DAYA'), '11010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM B')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM C')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM D')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM E')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM F')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM G')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM H')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM I')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM J')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BALIK PULAU')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BAYAN LEPAS')&(mukim_district_state['district']=='BARAT DAYA'), '11900', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PULAU PINANG')&(mukim_district_state['mukim'].notnull())]
mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1577014144.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13700', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1577014144.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_dist

postcode      271
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [40]:
# kubang pasu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AH')&(mukim_district_state['state']=='KEDAH'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BINJAL')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELONG')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT TINGGI')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HUSBA')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JERLUN')&(mukim_district_state['state']=='KEDAH'), '06150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEPELU')&(mukim_district_state['state']=='KEDAH'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUBANG PASU')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NAGA')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG PERAHU')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERING')&(mukim_district_state['state']=='KEDAH'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUTAT')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SANGLANG')&(mukim_district_state['state']=='KEDAH'), '06150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI LAKA')&(mukim_district_state['state']=='KEDAH'), '06010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMIN')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='WANG TEPUS')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR CHANGLUN')&(mukim_district_state['state']=='KEDAH'), '06010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANDAR DARULAMAN')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JERAM')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JITRA')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MALAU')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PELUBANG')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TUNJANG')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JITRA')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KODIANG')&(mukim_district_state['state']=='KEDAH'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TUNJANG')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PADANG SERA')&(mukim_district_state['state']=='KEDAH'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KEPALA BATAS')&(mukim_district_state['state']=='KEDAH'), '06200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BUKIT KAYU HITAM')&(mukim_district_state['state']=='KEDAH'), '06050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AYER HITAM')&(mukim_district_state['state']=='KEDAH'), '06150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA SANGLANG')&(mukim_district_state['state']=='KEDAH'), '06150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SANGLANG')&(mukim_district_state['state']=='KEDAH'), '06150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KERPAN')&(mukim_district_state['state']=='KEDAH'), '06150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SINTOK')&(mukim_district_state['state']=='KEDAH'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN NAPOH')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI KOROK')&(mukim_district_state['state']=='KEDAH'), '06150', mukim_district_state['postcode'])

#padang terap
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG TUNGGANG KANAN')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG TUNGGANG KIRI')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELIMBING KANAN')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELIMBING KIRI')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KURONG HITAM')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG TERNAK')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG TERAP KANAN')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG TERAP KIRI')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEKAI')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TOLAK')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEDU')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA NERANG')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN NAKA')&(mukim_district_state['state']=='KEDAH'), '06350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DURIAN BURUNG')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LUBOK MERBAU')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT TEMBAGA')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PADANG SANAI')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPUNG TANJUNG')&(mukim_district_state['state']=='KEDAH'), '06300', mukim_district_state['postcode'])

#kuala muda 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT MERIAM')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKULA')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMELING')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOI KIRI')&(mukim_district_state['state']=='KEDAH'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUJANG')&(mukim_district_state['state']=='KEDAH'), '08100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GURUN')&(mukim_district_state['state']=='KEDAH'), '08300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HAJI KUDONG')&(mukim_district_state['state']=='KEDAH'), '08500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KOTA')&(mukim_district_state['state']=='KEDAH'), '08500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MERBOK')&(mukim_district_state['state']=='KEDAH'), '08400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PINANG TUNGGAL')&(mukim_district_state['state']=='KEDAH'), '08000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RANTAU PANJANG')&(mukim_district_state['state']=='KEDAH'), '08500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIDAM KIRI')&(mukim_district_state['state']=='KEDAH'), '09400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIMPOR')&(mukim_district_state['state']=='KEDAH'), '08500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI PASIR')&(mukim_district_state['state']=='KEDAH'), '08000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI PETANI')&(mukim_district_state['state']=='KEDAH'), '08000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR AMAN JAYA')&(mukim_district_state['state']=='KEDAH'), '08000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BEDONG')&(mukim_district_state['state']=='KEDAH'), '08100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI LALANG')&(mukim_district_state['state']=='KEDAH'), '08100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TIKAM BATU')&(mukim_district_state['state']=='KEDAH'), '08600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JENIANG')&(mukim_district_state['state']=='KEDAH'), '08700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KOTA KUALA MUDA')&(mukim_district_state['state']=='KEDAH'), '08500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJUNG DAWAI')&(mukim_district_state['state']=='KEDAH'), '08200', mukim_district_state['postcode'])

#kota setar
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ALOR MALAI')&(mukim_district_state['state']=='KEDAH'), '05100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ALOR MERAH')&(mukim_district_state['state']=='KEDAH'), '05250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ANAK BUKIT')&(mukim_district_state['state']=='KEDAH'), '06550', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT PINANG')&(mukim_district_state['state']=='KEDAH'), '06720', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DERGA')&(mukim_district_state['state']=='KEDAH'), '05400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GUNONG')&(mukim_district_state['state']=='KEDAH'), '06570', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KOTA SETAR')&(mukim_district_state['state']=='KEDAH'), '05675', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HUTAN KAMPONG')&(mukim_district_state['state']=='KEDAH'), '05350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KANGKONG')&(mukim_district_state['state']=='KEDAH'), '06650', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KEDAH')&(mukim_district_state['state']=='KEDAH'), '06600', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUBANG ROTAN')&(mukim_district_state['state']=='KEDAH'), '06250', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LANGGAR')&(mukim_district_state['state']=='KEDAH'), '06500', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LENGKUAS')&(mukim_district_state['state']=='KEDAH'), '05350', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LEPAI')&(mukim_district_state['state']=='KEDAH'), '05350', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LIMBONG')&(mukim_district_state['state']=='KEDAH'), '06500', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MERGONG')&(mukim_district_state['state']=='KEDAH'), '05150', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG HANG')&(mukim_district_state['state']=='KEDAH'), '06250', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG LALANG')&(mukim_district_state['state']=='KEDAH'), '06250', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGKALAN KUNDOR')&(mukim_district_state['state']=='KEDAH'), '05400', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUMPONG')&(mukim_district_state['state']=='KEDAH'), '05250', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SALA KECHIK')&(mukim_district_state['state']=='KEDAH'), '06650', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI BAHARU')&(mukim_district_state['state']=='KEDAH'), '06250', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TAJAR')&(mukim_district_state['state']=='KEDAH'), '06500', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEBENGAU')&(mukim_district_state['state']=='KEDAH'), '06600', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELAGA MAS')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOK CHENGAI')&(mukim_district_state['state']=='KEDAH'), '06600', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TITI GAJAH')&(mukim_district_state['state']=='KEDAH'), '06550', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOK KECHAI')&(mukim_district_state['state']=='KEDAH'), '06600', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ALOR SETAR')&(mukim_district_state['state']=='KEDAH'), '05100', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ANAK BUKIT')&(mukim_district_state['state']=='KEDAH'), '05100', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KEDAH')&(mukim_district_state['state']=='KEDAH'), '06600', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ALOR MERAH')&(mukim_district_state['state']=='KEDAH'), '05250', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BUKIT PINANG')&(mukim_district_state['state']=='KEDAH'), '06200', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LANGGAR')&(mukim_district_state['state']=='KEDAH'), '06500', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SIMPANG EMPAT')&(mukim_district_state['state']=='KEDAH'), '06250', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN ALOR JANGGUS')&(mukim_district_state['state']=='KEDAH'), '06250', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KOTA SARANG SEMUT')&(mukim_district_state['state']=='KEDAH'), '06800', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GUNUNG')&(mukim_district_state['state']=='KEDAH'), '05150', mukim_district_state['postcode']) 

# langkawi
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER HANGAT')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BOHOR')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEDAWANG')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUAH')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG MASIRAT')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU MELAKA')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUAH')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PADANG LALANG')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TELOK DATAI')&(mukim_district_state['state']=='KEDAH'), '07000', mukim_district_state['postcode']) 

# pokok sena
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DERANG')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GAJAH MATI')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JABI')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LESONG')&(mukim_district_state['state']=='KEDAH'), '06350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TUALANG')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT LADA')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR POKOK SENA')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KEBUN 500')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])

# yan
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DULANG')&(mukim_district_state['state']=='KEDAH'), '06900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SALA BESAR')&(mukim_district_state['state']=='KEDAH'), '06900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SINGKIR')&(mukim_district_state['state']=='KEDAH'), '06900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI DAUN')&(mukim_district_state['state']=='KEDAH'), '06800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='YAN')&(mukim_district_state['state']=='KEDAH'), '06900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GUAR CHEMPEDAK')&(mukim_district_state['state']=='KEDAH'), '08800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR YAN')&(mukim_district_state['state']=='KEDAH'), '06900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG TIGA SUNGAI LIMAU')&(mukim_district_state['state']=='KEDAH'), '06800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI LIMAU DALAM')&(mukim_district_state['state']=='KEDAH'), '06900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TEROI')&(mukim_district_state['state']=='KEDAH'), '06900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SINGKIR')&(mukim_district_state['state']=='KEDAH'), '06900', mukim_district_state['postcode'])

# sik
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JENERI')&(mukim_district_state['state']=='KEDAH'), '08700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIK')&(mukim_district_state['state']=='KEDAH'), '08340', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SOK')&(mukim_district_state['state']=='KEDAH'), '08210', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOI')&(mukim_district_state['state']=='KEDAH'), '08340', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SIK')&(mukim_district_state['state']=='KEDAH'), '08340', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU LAMA SIK')&(mukim_district_state['state']=='KEDAH'), '08210', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GULAU')&(mukim_district_state['state']=='KEDAH'), '08210', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GAJAH PUTEH')&(mukim_district_state['state']=='KEDAH'), '08320', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHAROK PADANG')&(mukim_district_state['state']=='KEDAH'), '08200', mukim_district_state['postcode'])


# pendang
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER PUTEH')&(mukim_district_state['state']=='KEDAH'), '06700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT RAYA')&(mukim_district_state['state']=='KEDAH'), '06700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GUAR KEPAYANG')&(mukim_district_state['state']=='KEDAH'), '06700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG KERBAU')&(mukim_district_state['state']=='KEDAH'), '06700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG PELIANG')&(mukim_district_state['state']=='KEDAH'), '06750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG PUSING')&(mukim_district_state['state']=='KEDAH'), '06700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RAMBAI')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TOBIAR')&(mukim_district_state['state']=='KEDAH'), '06700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PEDANG')&(mukim_district_state['state']=='KEDAH'), '06700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT JENUN')&(mukim_district_state['state']=='KEDAH'), '06720', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUBUR PANJANG')&(mukim_district_state['state']=='KEDAH'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANAH MERAH')&(mukim_district_state['state']=='KEDAH'), '06700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TOKAI')&(mukim_district_state['state']=='KEDAH'), '06650', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KOBAH')&(mukim_district_state['state']=='KEDAH'), '06800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPUNG BARU')&(mukim_district_state['state']=='KEDAH'), '06800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI TIANG')&(mukim_district_state['state']=='KEDAH'), '06800', mukim_district_state['postcode'])


# baling
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAKAI')&(mukim_district_state['state']=='KEDAH'), '09120', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BALING')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BONGOR')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUPANG')&(mukim_district_state['state']=='KEDAH'), '09200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAI')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIONG')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TAWAR')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOI KANAN')&(mukim_district_state['state']=='KEDAH'), '09310', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BALING')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KETIL')&(mukim_district_state['state']=='KEDAH'), '09300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUPANG')&(mukim_district_state['state']=='KEDAH'), '09200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPUNG BARU KEJAI')&(mukim_district_state['state']=='KEDAH'), '09120', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PULAI')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TAWAR')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA PEGANG')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PARIT PANJANG')&(mukim_district_state['state']=='KEDAH'), '09200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPUNG LALANG')&(mukim_district_state['state']=='KEDAH'), '09100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MALAU')&(mukim_district_state['state']=='KEDAH'), '09000', mukim_district_state['postcode'])


# kulim
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAGAN SENA')&(mukim_district_state['state']=='KEDAH'), '09000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JUNJONG')&(mukim_district_state['state']=='KEDAH'), '09000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KARANGAN')&(mukim_district_state['state']=='KEDAH'), '09410', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KELADI')&(mukim_district_state['state']=='KEDAH'), '09000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KULIM')&(mukim_district_state['state']=='KEDAH'), '09000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LUNAS')&(mukim_district_state['state']=='KEDAH'), '09600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MAHANG')&(mukim_district_state['state']=='KEDAH'), '09700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NAGA LILIT')&(mukim_district_state['state']=='KEDAH'), '09400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG CHINA')&(mukim_district_state['state']=='KEDAH'), '09600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG MEHA')&(mukim_district_state['state']=='KEDAH'), '09010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEDIM')&(mukim_district_state['state']=='KEDAH'), '09700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIDAM KANAN')&(mukim_district_state['state']=='KEDAH'), '09400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI SELUANG')&(mukim_district_state['state']=='KEDAH'), '09600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI ULAR')&(mukim_district_state['state']=='KEDAH'), '09000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TERAP')&(mukim_district_state['state']=='KEDAH'), '09800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KULIM')&(mukim_district_state['state']=='KEDAH'), '09000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LUNAS')&(mukim_district_state['state']=='KEDAH'), '09600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PADANG SERAI')&(mukim_district_state['state']=='KEDAH'), '09400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JUNJONG')&(mukim_district_state['state']=='KEDAH'), '09000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KARANGAN')&(mukim_district_state['state']=='KEDAH'), '09410', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LABU BESAR')&(mukim_district_state['state']=='KEDAH'), '09010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MAHANG')&(mukim_district_state['state']=='KEDAH'), '09700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MERBAU PULAS')&(mukim_district_state['state']=='KEDAH'), '09300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI KARANGAN')&(mukim_district_state['state']=='KEDAH'), '09410', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI KOB')&(mukim_district_state['state']=='KEDAH'), '09700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PADANG MEHA')&(mukim_district_state['state']=='KEDAH'), '09010', mukim_district_state['postcode'])

# bandar baharu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAGAN SAMAK')&(mukim_district_state['state']=='KEDAH'), '34950', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA SELAMA')&(mukim_district_state['state']=='KEDAH'), '09800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERMATANG PASIR')&(mukim_district_state['state']=='KEDAH'), '34950', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RELAU')&(mukim_district_state['state']=='KEDAH'), '14290', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERDANG')&(mukim_district_state['state']=='KEDAH'), '09810', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI BATU')&(mukim_district_state['state']=='KEDAH'), '09800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI KECHIL')&(mukim_district_state['state']=='KEDAH'), '34950', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANDAR BAHARU')&(mukim_district_state['state']=='KEDAH'), '34950', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SERDANG')&(mukim_district_state['state']=='KEDAH'), '09800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LUBUK BUNTAR')&(mukim_district_state['state']=='KEDAH'), '09800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SELAMA')&(mukim_district_state['state']=='KEDAH'), '09800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI KECIL ILIR')&(mukim_district_state['state']=='KEDAH'), '14390', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN RELAU')&(mukim_district_state['state']=='KEDAH'), '14290', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='KEDAH')&(mukim_district_state['mukim'].notnull())]
mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\2279664161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AH')&(mukim_district_state['state']=='KEDAH'), '06100', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\2279664161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].i

postcode      238
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [41]:
#print(len(mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERAK')]))
#pd.set_option('display.max_rows', None)
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERAK')]

# batang padang
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG PADANG')&(mukim_district_state['state']=='PERAK'), '35000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SLIM')&(mukim_district_state['state']=='PERAK'), '35800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR CHENDERIAN')&(mukim_district_state['state']=='PERAK'), '35350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG MALIM')&(mukim_district_state['state']=='PERAK'), '35600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHENDERIANG')&(mukim_district_state['state']=='PERAK'), '35300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU BERNAM BARAT')&(mukim_district_state['state']=='PERAK'), '35900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU BERNAM TIMOR')&(mukim_district_state['state']=='PERAK'), '35900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGKAI')&(mukim_district_state['state']=='PERAK'), '35600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BIDOR')&(mukim_district_state['state']=='PERAK'), '35000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGKAI')&(mukim_district_state['state']=='PERAK'), '35600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAPAH')&(mukim_district_state['state']=='PERAK'), '35000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TEMOH')&(mukim_district_state['state']=='PERAK'), '35350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AYER KUNING')&(mukim_district_state['state']=='PERAK'), '31900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BANIR')&(mukim_district_state['state']=='PERAK'), '35400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BANTANG')&(mukim_district_state['state']=='PERAK'), '35800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BIKAM')&(mukim_district_state['state']=='PERAK'), '35600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SLIM')&(mukim_district_state['state']=='PERAK'), '35800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI LESONG')&(mukim_district_state['state']=='PERAK'), '31900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TAPAH ROAD')&(mukim_district_state['state']=='PERAK'), '35400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TEMOH STATION')&(mukim_district_state['state']=='PERAK'), '35350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TEROLAK')&(mukim_district_state['state']=='PERAK'), '35700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BIDOR')&(mukim_district_state['state']=='PERAK'), '35000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG MALIM')&(mukim_district_state['state']=='PERAK'), '35900', mukim_district_state['postcode'])

# manjung
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERUAS')&(mukim_district_state['state']=='PERAK'), '32700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGKALAN BAHARU')&(mukim_district_state['state']=='PERAK'), '34900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LEKIR')&(mukim_district_state['state']=='PERAK'), '32000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LUMUT')&(mukim_district_state['state']=='PERAK'), '32200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SITIAWAN')&(mukim_district_state['state']=='PERAK'), '32000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LUMUT')&(mukim_district_state['state']=='PERAK'), '32200', mukim_district_state['postcode'])   
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AYER TAWAR')&(mukim_district_state['state']=='PERAK'), '32400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BERUAS')&(mukim_district_state['state']=='PERAK'), '32700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHANGKAT KERUING')&(mukim_district_state['state']=='PERAK'), '32500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DAMAR LAUT')&(mukim_district_state['state']=='PERAK'), '32200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPONG BAHARU')&(mukim_district_state['state']=='PERAK'), '32200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPONG KOH')&(mukim_district_state['state']=='PERAK'), '32000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPONG SITIAWAN')&(mukim_district_state['state']=='PERAK'), '32000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PANGKOR')&(mukim_district_state['state']=='PERAK'), '32300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PANTAI REMIS')&(mukim_district_state['state']=='PERAK'), '34900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PASIR BOGAK')&(mukim_district_state['state']=='PERAK'), '32300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GURNEY')&(mukim_district_state['state']=='PERAK'), '32000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PANGKALAN BAHARU')&(mukim_district_state['state']=='PERAK'), '34900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SEGARI')&(mukim_district_state['state']=='PERAK'), '32200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SITIAWAN')&(mukim_district_state['state']=='PERAK'), '32000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI PINANG KECHIL')&(mukim_district_state['state']=='PERAK'), '32300', mukim_district_state['postcode'])

# kinta
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELANJA')&(mukim_district_state['state']=='PERAK'), '32610', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG TUALANG')&(mukim_district_state['state']=='PERAK'), '31800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR IPOH (U)')&(mukim_district_state['state']=='PERAK'), '30020', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR IPOH (S)')&(mukim_district_state['state']=='PERAK'), '31350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MENGELEMBU')&(mukim_district_state['state']=='PERAK'), '31450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PAPAN')&(mukim_district_state['state']=='PERAK'), '31550', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEPUTEH')&(mukim_district_state['state']=='PERAK'), '31560', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI RAYA')&(mukim_district_state['state']=='PERAK'), '31300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG RAMBUTAN')&(mukim_district_state['state']=='PERAK'), '31250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TERONOH')&(mukim_district_state['state']=='PERAK'), '31750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU KINTA')&(mukim_district_state['state']=='PERAK'), '31150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI RAIA')&(mukim_district_state['state']=='PERAK'), '31300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI TERAP')&(mukim_district_state['state']=='PERAK'), '31000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU GAJAH')&(mukim_district_state['state']=='PERAK'), '31000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR CHEMOR')&(mukim_district_state['state']=='PERAK'), '31200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JELAPANG')&(mukim_district_state['state']=='PERAK'), '30020 ', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LAHAT')&(mukim_district_state['state']=='PERAK'), '31500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PUSING')&(mukim_district_state['state']=='PERAK'), '31550', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAMBUN')&(mukim_district_state['state']=='PERAK'), '31400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KANTHAN')&(mukim_district_state['state']=='PERAK'), '31200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG PULAI')&(mukim_district_state['state']=='PERAK'), '31300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJONG TUALANG')&(mukim_district_state['state']=='PERAK'), '31800', mukim_district_state['postcode'])

# kerian
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERIAH')&(mukim_district_state['state']=='PERAK'), '34300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GUNONG SEMANGGOL')&(mukim_district_state['state']=='PERAK'), '34400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELINSING')&(mukim_district_state['state']=='PERAK'), '34400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG PIANDANG')&(mukim_district_state['state']=='PERAK'), '34350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAGAN SERAI')&(mukim_district_state['state']=='PERAK'), '34300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAGAN TIANG')&(mukim_district_state['state']=='PERAK'), '34250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KURAU')&(mukim_district_state['state']=='PERAK'), '34350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PARIT BUNTAR')&(mukim_district_state['state']=='PERAK'), '34200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BAGAN SERAI')&(mukim_district_state['state']=='PERAK'), '34300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KURAU')&(mukim_district_state['state']=='PERAK'), '34350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PARIT BUNTAR')&(mukim_district_state['state']=='PERAK'), '34200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT MERAH')&(mukim_district_state['state']=='PERAK'), '34400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JALAN BARU')&(mukim_district_state['state']=='PERAK'), '34300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG LIMA')&(mukim_district_state['state']=='PERAK'), '34200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI GEDONG')&(mukim_district_state['state']=='PERAK'), '34300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJONG PIANDANG')&(mukim_district_state['state']=='PERAK'), '34250', mukim_district_state['postcode'])

# hilir perak
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAGAN DATUK')&(mukim_district_state['state']=='PERAK'), '36100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHANGKAT JONG')&(mukim_district_state['state']=='PERAK'), '36700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RUNGKUP')&(mukim_district_state['state']=='PERAK'), '36200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI MANIK')&(mukim_district_state['state']=='PERAK'), '36750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOK BARU')&(mukim_district_state['state']=='PERAK'), '36300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DURIAN SEBATANG')&(mukim_district_state['state']=='PERAK'), '36000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HUTAN MELINTANG')&(mukim_district_state['state']=='PERAK'), '36400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LABU KUBONG')&(mukim_district_state['state']=='PERAK'), '36600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI DURIAN')&(mukim_district_state['state']=='PERAK'), '36000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TELUK INTAN')&(mukim_district_state['state']=='PERAK'), '36000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BAGAN DATUK')&(mukim_district_state['state']=='PERAK'), '36100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATAK RABIT')&(mukim_district_state['state']=='PERAK'), '36000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU DUA PULOH')&(mukim_district_state['state']=='PERAK'), '36200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHIKUS')&(mukim_district_state['state']=='PERAK'), '36750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DEGONG')&(mukim_district_state['state']=='PERAK'), '36700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN HUTAN MELINTANG')&(mukim_district_state['state']=='PERAK'), '36400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JENDARATA')&(mukim_district_state['state']=='PERAK'), '36400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPUNG SUNGAI HAJI MOHAMED')&(mukim_district_state['state']=='PERAK'), '36200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LANGKAP')&(mukim_district_state['state']=='PERAK'), '36700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SELEKOH')&(mukim_district_state['state']=='PERAK'), '36200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG EMPAT')&(mukim_district_state['state']=='PERAK'), '36400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANT TIGA (RUNGKUP)')&(mukim_district_state['state']=='PERAK'), '36200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI SUMUN')&(mukim_district_state['state']=='PERAK'), '36300', mukim_district_state['postcode'])

# hulu perak
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELUKAR SEMANG')&(mukim_district_state['state']=='PERAK'), '33200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELUM')&(mukim_district_state['state']=='PERAK'), '33300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DURIAN PIPIT')&(mukim_district_state['state']=='PERAK'), '33030', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GRIK')&(mukim_district_state['state']=='PERAK'), '33300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERUNAI')&(mukim_district_state['state']=='PERAK'), '33310', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KENERING')&(mukim_district_state['state']=='PERAK'), '33410', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMELONG')&(mukim_district_state['state']=='PERAK'), '33400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMENGOR')&(mukim_district_state['state']=='PERAK'), '33300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LAWN')&(mukim_district_state['state']=='PERAK'), '33410', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGKALAN HULU')&(mukim_district_state['state']=='PERAK'), '33100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LENGGONG')&(mukim_district_state['state']=='PERAK'), '33400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GERIK')&(mukim_district_state['state']=='PERAK'), '33300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KELIAN INTAN')&(mukim_district_state['state']=='PERAK'), '33100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PENGKALAN HULU')&(mukim_district_state['state']=='PERAK'), '33100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LENGGONG')&(mukim_district_state['state']=='PERAK'), '33400', mukim_district_state['postcode'])

# perak tengah
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR')&(mukim_district_state['state']=='PERAK'), '36800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELANJA')&(mukim_district_state['state']=='PERAK'), '32800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JAYA BARU')&(mukim_district_state['state']=='PERAK'), '32000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAMPONG GAJAH')&(mukim_district_state['state']=='PERAK'), '36800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LAMBOR KIRI')&(mukim_district_state['state']=='PERAK'), '32600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR PANJANG HULU')&(mukim_district_state['state']=='PERAK'), '36800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BOTA')&(mukim_district_state['state']=='PERAK'), '32600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KOTA SETIA')&(mukim_district_state['state']=='PERAK'), '36810', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LAMBOR KANAN')&(mukim_district_state['state']=='PERAK'), '32900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LAYANG LAYANG')&(mukim_district_state['state']=='PERAK'), '32800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR SALAK')&(mukim_district_state['state']=='PERAK'), '36800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU TIGA')&(mukim_district_state['state']=='PERAK'), '36800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANDAR SERI ISKANDAR')&(mukim_district_state['state']=='PERAK'), '32610', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BOTA KANAN')&(mukim_district_state['state']=='PERAK'), '32600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPONG BULOH AKAR')&(mukim_district_state['state']=='PERAK'), '32800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KOTA SETIA')&(mukim_district_state['state']=='PERAK'), '36810', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PARIT')&(mukim_district_state['state']=='PERAK'), '32900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJONG BELANJA')&(mukim_district_state['state']=='PERAK'), '32800', mukim_district_state['postcode'])

# kuala kangsar
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHENGAR GALAH')&(mukim_district_state['state']=='PERAK'), '33500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAMPONG BUAYA')&(mukim_district_state['state']=='PERAK'), '33700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KOTA LAMA KANAN')&(mukim_district_state['state']=='PERAK'), '33040', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KOTA LAMA KIRI')&(mukim_district_state['state']=='PERAK'), '33000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LUBOK MERBAU')&(mukim_district_state['state']=='PERAK'), '33000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU KAMIRI')&(mukim_district_state['state']=='PERAK'), '31100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SAIONG')&(mukim_district_state['state']=='PERAK'), '33040', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SENGGANG')&(mukim_district_state['state']=='PERAK'), '33000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI SIPUT')&(mukim_district_state['state']=='PERAK'), '33000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KANGSAR')&(mukim_district_state['state']=='PERAK'), '33000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI SIPUT')&(mukim_district_state['state']=='PERAK'), '31100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GUNONG PONDOK')&(mukim_district_state['state']=='PERAK'), '33700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JERLUN')&(mukim_district_state['state']=='PERAK'), '33000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KARAI')&(mukim_district_state['state']=='PERAK'), '31050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KATI')&(mukim_district_state['state']=='PERAK'), '33020', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LUBOK MERBAU')&(mukim_district_state['state']=='PERAK'), '33010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MANONG')&(mukim_district_state['state']=='PERAK'), '33800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PADANG RENGAS')&(mukim_district_state['state']=='PERAK'), '33700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SALAK')&(mukim_district_state['state']=='PERAK'), '31050', mukim_district_state['postcode'])

# larut matang
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ASAM KUMBANG')&(mukim_district_state['state']=='PERAK'), '34000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU KURAU')&(mukim_district_state['state']=='PERAK'), '34500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT GANTANG')&(mukim_district_state['state']=='PERAK'), '34030', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JEBONG')&(mukim_district_state['state']=='PERAK'), '34750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAMUNTING')&(mukim_district_state['state']=='PERAK'), '34600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGKALAN AOR')&(mukim_district_state['state']=='PERAK'), '34000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIMPANG')&(mukim_district_state['state']=='PERAK'), '34000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI LIMAU')&(mukim_district_state['state']=='PERAK'), '34400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI TINGGI')&(mukim_district_state['state']=='PERAK'), '34800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TERONG')&(mukim_district_state['state']=='PERAK'), '34800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TUPAI')&(mukim_district_state['state']=='PERAK'), '34000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KAMUNTING')&(mukim_district_state['state']=='PERAK'), '34600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA SEPETANG')&(mukim_district_state['state']=='PERAK'), '34650', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MATANG')&(mukim_district_state['state']=='PERAK'), '34700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAIPING')&(mukim_district_state['state']=='PERAK'), '34000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU KURAU')&(mukim_district_state['state']=='PERAK'), '34500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHANGKAT JERING')&(mukim_district_state['state']=='PERAK'), '34850', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PONDOK TANJONG')&(mukim_district_state['state']=='PERAK'), '34010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG')&(mukim_district_state['state']=='PERAK'), '34000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TERUNG')&(mukim_district_state['state']=='PERAK'), '34800', mukim_district_state['postcode'])

# selama
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU IJOK')&(mukim_district_state['state']=='PERAK'), '34510', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU SELAMA')&(mukim_district_state['state']=='PERAK'), '34140', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELAMA')&(mukim_district_state['state']=='PERAK'), '34100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SELAMA')&(mukim_district_state['state']=='PERAK'), '34100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN RANTAU PANJANG')&(mukim_district_state['state']=='PERAK'), '34140', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI BAYUR')&(mukim_district_state['state']=='PERAK'), '34140', mukim_district_state['postcode'])

# kampar
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAMPAR')&(mukim_district_state['state']=='PERAK'), '31900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEJA')&(mukim_district_state['state']=='PERAK'), '31600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GOPENG')&(mukim_district_state['state']=='PERAK'), '31600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KAMPAR')&(mukim_district_state['state']=='PERAK'), '31900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KOTA BAHARU')&(mukim_district_state['state']=='PERAK'), '31600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MALIM NAWAR')&(mukim_district_state['state']=='PERAK'), '31700', mukim_district_state['postcode'])


mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERAK')&(mukim_district_state['mukim'].notnull())]	
mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\3297437906.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG PADANG')&(mukim_district_state['state']=='PERAK'), '35000', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\3297437906.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['p

postcode      198
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [42]:
# print(len(mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SELANGOR')]))
# mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SELANGOR')]

# klang
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KLANG')&(mukim_district_state['state']=='SELANGOR'), '41400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KLANG')&(mukim_district_state['state']=='SELANGOR'), '41000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PORT SWETTENHAM')&(mukim_district_state['state']=='SELANGOR'), '42000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SHAH ALAM')&(mukim_district_state['state']=='SELANGOR'), '40000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAPAR')&(mukim_district_state['state']=='SELANGOR'), '42200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SULTAN SULAIMAN')&(mukim_district_state['state']=='SELANGOR'), '42000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT KEMUNING')&(mukim_district_state['state']=='SELANGOR'), '40460', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAPAR')&(mukim_district_state['state']=='SELANGOR'), '42200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MERU')&(mukim_district_state['state']=='SELANGOR'), '41050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TELOK MENEGUN')&(mukim_district_state['state']=='SELANGOR'), '40460', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU EMPAT')&(mukim_district_state['state']=='SELANGOR'), '42100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PANDAMARAN')&(mukim_district_state['state']=='SELANGOR'), '42000', mukim_district_state['postcode'])

# kuala langat
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JUGRA')&(mukim_district_state['state']=='SELANGOR'), '42960', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG DUABELAS')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG SEPAT')&(mukim_district_state['state']=='SELANGOR'), '42800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KELANANG')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MORIB')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOK PANGLIMA GARANG')&(mukim_district_state['state']=='SELANGOR'), '42500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANTING')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JENJAROM')&(mukim_district_state['state']=='SELANGOR'), '42600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SIJANGKANG')&(mukim_district_state['state']=='SELANGOR'), '42500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TELOK PANGLIMA GARANG')&(mukim_district_state['state']=='SELANGOR'), '42500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT CHANGGANG')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHODOI')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JENJAROM')&(mukim_district_state['state']=='SELANGOR'), '42600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KANCHONG')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KANCHONG DARAT')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KELANANG BATU ENAM')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MORIB')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIJANGKANG')&(mukim_district_state['state']=='SELANGOR'), '42500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PERMATANG PASIR')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG MORIB')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI MANGGIS')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI RABA')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJONG DUABELAS')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TELOK DATOK')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TONGKAH')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TELOK')&(mukim_district_state['state']=='SELANGOR'), '42700', mukim_district_state['postcode'])

# kuala selangor
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='API-API')&(mukim_district_state['state']=='SELANGOR'), '45700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='IJOK')&(mukim_district_state['state']=='SELANGOR'), '45609', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JERAM')&(mukim_district_state['state']=='SELANGOR'), '45800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA SELANGOR')&(mukim_district_state['state']=='SELANGOR'), '45000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASANGAN')&(mukim_district_state['state']=='SELANGOR'), '45600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG KARANG')&(mukim_district_state['state']=='SELANGOR'), '45500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BESTARI JAYA')&(mukim_district_state['state']=='SELANGOR'), '45600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA SELANGOR')&(mukim_district_state['state']=='SELANGOR'), '45000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG KARANG')&(mukim_district_state['state']=='SELANGOR'), '45500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN ASAM JAWA')&(mukim_district_state['state']=='SELANGOR'), '45700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT ROTAN')&(mukim_district_state['state']=='SELANGOR'), '45620', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JERAM')&(mukim_district_state['state']=='SELANGOR'), '45800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPONG KUANTAN')&(mukim_district_state['state']=='SELANGOR'), '45600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA SUNGAI BULOH')&(mukim_district_state['state']=='SELANGOR'), '45800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PASIR PENAMPANG')&(mukim_district_state['state']=='SELANGOR'), '45000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG TIGA')&(mukim_district_state['state']=='SELANGOR'), '45600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJONG KARANG')&(mukim_district_state['state']=='SELANGOR'), '45500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKTI BELIMBING')&(mukim_district_state['state']=='SELANGOR'), '45000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT TALANG')&(mukim_district_state['state']=='SELANGOR'), '45500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMBUNG BARU HULU TIRAM BURUK')&(mukim_district_state['state']=='SELANGOR'), '45500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PARTI MAHANG')&(mukim_district_state['state']=='SELANGOR'), '45800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG TIGA IJOK')&(mukim_district_state['state']=='SELANGOR'), '45700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI SEMBILANG')&(mukim_district_state['state']=='SELANGOR'), '45800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TAMAN PKNS')&(mukim_district_state['state']=='SELANGOR'), '45000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TAMBAK JAWA')&(mukim_district_state['state']=='SELANGOR'), '45600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BESTARI JAYA')&(mukim_district_state['state']=='SELANGOR'), '45600', mukim_district_state['postcode'])

# sabak bernam
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAGAN NAKHODA OMAR')&(mukim_district_state['state']=='SELANGOR'), '45200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PANCANG BEDENA')&(mukim_district_state['state']=='SELANGOR'), '45300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR PANJANG')&(mukim_district_state['state']=='SELANGOR'), '45400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SABAK')&(mukim_district_state['state']=='SELANGOR'), '45200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI PANJANG')&(mukim_district_state['state']=='SELANGOR'), '45300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BAGAN TERAP')&(mukim_district_state['state']=='SELANGOR'), '45209', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PARIT ENAM')&(mukim_district_state['state']=='SELANGOR'), '45300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PARIT SEMBILAN')&(mukim_district_state['state']=='SELANGOR'), '45300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SEKINCHAN')&(mukim_district_state['state']=='SELANGOR'), '45400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SABAK')&(mukim_district_state['state']=='SELANGOR'), '45200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI AIR TAWAR')&(mukim_district_state['state']=='SELANGOR'), '45200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI SEPINTAS')&(mukim_district_state['state']=='SELANGOR'), '45200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BAGAN NAKHODA OMAR')&(mukim_district_state['state']=='SELANGOR'), '45200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PARIT BARU')&(mukim_district_state['state']=='SELANGOR'), '45400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PASIR PANJANG')&(mukim_district_state['state']=='SELANGOR'), '45400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SEKINCHAN SITE A')&(mukim_district_state['state']=='SELANGOR'), '45400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI BESAR')&(mukim_district_state['state']=='SELANGOR'), '45300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI HAJI DORANI')&(mukim_district_state['state']=='SELANGOR'), '45300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI NIBONG')&(mukim_district_state['state']=='SELANGOR'), '45400', mukim_district_state['postcode'])

# ulu langat
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR CHERAS')&(mukim_district_state['state']=='SELANGOR'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU LANGAT')&(mukim_district_state['state']=='SELANGOR'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BALAKONG')&(mukim_district_state['state']=='SELANGOR'), '43300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU 9, CHERAS')&(mukim_district_state['state']=='SELANGOR'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERANANG')&(mukim_district_state['state']=='SELANGOR'), '43700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU LANGAT')&(mukim_district_state['state']=='SELANGOR'), '43100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU SEMENYIH')&(mukim_district_state['state']=='SELANGOR'), '43500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAJANG')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMENYIH')&(mukim_district_state['state']=='SELANGOR'), '43500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KAJANG')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEMENYIH')&(mukim_district_state['state']=='SELANGOR'), '43500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR AMPANG')&(mukim_district_state['state']=='SELANGOR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU 18, SEMENYIH')&(mukim_district_state['state']=='SELANGOR'), '43100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU 26, BERANANG')&(mukim_district_state['state']=='SELANGOR'), '43700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BERANANG')&(mukim_district_state['state']=='SELANGOR'), '43700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KACAU')&(mukim_district_state['state']=='SELANGOR'), '43500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TARUN')&(mukim_district_state['state']=='SELANGOR'), '43500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BANGI LAMA')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU 23, SUNGAI LALANG')&(mukim_district_state['state']=='SELANGOR'), '43500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU 26, BERANANG')&(mukim_district_state['state']=='SELANGOR'), '43700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT SUNGAI RAYA')&(mukim_district_state['state']=='SELANGOR'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHERAS')&(mukim_district_state['state']=='SELANGOR'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DESA RAYA')&(mukim_district_state['state']=='SELANGOR'), '43100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DUSUN TUA ULU LANGAT')&(mukim_district_state['state']=='SELANGOR'), '43100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAJANG')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPONG PASIR, BATU 14, SEMENYIH')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPONG SUNGAI TANGKAS')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN RUMAH MURAH SUNGAI LUI')&(mukim_district_state['state']=='SELANGOR'), '43100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SEMENYIH')&(mukim_district_state['state']=='SELANGOR'), '43500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG BALAK')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SRI NANDING')&(mukim_district_state['state']=='SELANGOR'), '43100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI KEMBONG BERANANG')&(mukim_district_state['state']=='SELANGOR'), '43700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI LUI')&(mukim_district_state['state']=='SELANGOR'), '43100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI MAKAU')&(mukim_district_state['state']=='SELANGOR'), '43100', mukim_district_state['postcode'])

# ulu selangor
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BULUH TELOR')&(mukim_district_state['state']=='SELANGOR'), '44000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU YAM')&(mukim_district_state['state']=='SELANGOR'), '44300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERETAK')&(mukim_district_state['state']=='SELANGOR'), '44000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI GUMUT')&(mukim_district_state['state']=='SELANGOR'), '44000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI TINGGI')&(mukim_district_state['state']=='SELANGOR'), '44200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU YAM')&(mukim_district_state['state']=='SELANGOR'), '44300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU YAM BAHARU')&(mukim_district_state['state']=='SELANGOR'), '44300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KUBU BAHARU')&(mukim_district_state['state']=='SELANGOR'), '44000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI CHIK')&(mukim_district_state['state']=='SELANGOR'), '44300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG KALI')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AMPANG PECHAH')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU BERNAM')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KALUMPANG')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERLING')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KALUMPANG')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RASA')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERENDAH')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KALUMPANG')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR RASA')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SERENDAH')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATANG KALI')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU BERNAM I')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU BERNAM II')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KERLING')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PERETAK')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG SUNGAI CHOH')&(mukim_district_state['state']=='SELANGOR'), '', mukim_district_state['postcode'])

# petaling
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SHAH ALAM')&(mukim_district_state['state']=='SELANGOR'), '40250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR DAMANSARA')&(mukim_district_state['state']=='SELANGOR'), '46350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GLENMARIE')&(mukim_district_state['state']=='SELANGOR'), '40250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PETALING JAYA SELATAN')&(mukim_district_state['state']=='SELANGOR'), '46000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SRI DAMANSARA')&(mukim_district_state['state']=='SELANGOR'), '52200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT RAJA')&(mukim_district_state['state']=='SELANGOR'), '40170', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DAMANSARA')&(mukim_district_state['state']=='SELANGOR'), '52200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI BULOH')&(mukim_district_state['state']=='SELANGOR'), '47000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SAUJANA')&(mukim_district_state['state']=='SELANGOR'), '47000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUBANG JAYA')&(mukim_district_state['state']=='SELANGOR'), '47500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNWAY')&(mukim_district_state['state']=='SELANGOR'), '47500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU TIGA')&(mukim_district_state['state']=='SELANGOR'), '40150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MERBAU SEMPAK')&(mukim_district_state['state']=='SELANGOR'), '40160', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PUCHONG')&(mukim_district_state['state']=='SELANGOR'), '47100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SERDANG')&(mukim_district_state['state']=='SELANGOR'), '43300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI BULOH')&(mukim_district_state['state']=='SELANGOR'), '47000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CEMPAKA')&(mukim_district_state['state']=='SELANGOR'), '47301', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN COUNTRY HEIGHT')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DESA PUCHONG')&(mukim_district_state['state']=='SELANGOR'), '47150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN HICOM')&(mukim_district_state['state']=='SELANGOR'), '40150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAYU ARA')&(mukim_district_state['state']=='SELANGOR'), '47400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KINRARA')&(mukim_district_state['state']=='SELANGOR'), '47180', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BARU HICOM')&(mukim_district_state['state']=='SELANGOR'), '40400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BARU SUBANG')&(mukim_district_state['state']=='SELANGOR'), '47200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BARU SUNGAI BESI')&(mukim_district_state['state']=='SELANGOR'), '43300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BARU SUNGAI BULOH')&(mukim_district_state['state']=='SELANGOR'), '40160', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BARU PENAGA')&(mukim_district_state['state']=='SELANGOR'), '43400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PUCHONG JAYA')&(mukim_district_state['state']=='SELANGOR'), '47170', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PUCHONG PERDANA')&(mukim_district_state['state']=='SELANGOR'), '47150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUBANG')&(mukim_district_state['state']=='SELANGOR'), '40150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUBANG JAYA')&(mukim_district_state['state']=='SELANGOR'), '47500', mukim_district_state['postcode'])

# gombak
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GOMBAK SETIA')&(mukim_district_state['state']=='SELANGOR'), '53100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU KELANG')&(mukim_district_state['state']=='SELANGOR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KEPONG')&(mukim_district_state['state']=='SELANGOR'), '52100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUNDANG')&(mukim_district_state['state']=='SELANGOR'), '48050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI PUSU')&(mukim_district_state['state']=='SELANGOR'), '53000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RAWANG')&(mukim_district_state['state']=='SELANGOR'), '48000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU ARANG')&(mukim_district_state['state']=='SELANGOR'), '48000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR RAWANG')&(mukim_district_state['state']=='SELANGOR'), '48000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SELAYANG')&(mukim_district_state['state']=='SELANGOR'), '68100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI BULOH')&(mukim_district_state['state']=='SELANGOR'), '47000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU 20')&(mukim_district_state['state']=='SELANGOR'), '48200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUANG')&(mukim_district_state['state']=='SELANGOR'), '47000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MIMALAND')&(mukim_district_state['state']=='SELANGOR'), '53100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PENGKALAN KUNDANG')&(mukim_district_state['state']=='SELANGOR'), '48020', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI BULOH')&(mukim_district_state['state']=='SELANGOR'), '47000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TEMPLER')&(mukim_district_state['state']=='SELANGOR'), '48000', mukim_district_state['postcode'])

# sepang
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DENGKIL')&(mukim_district_state['state']=='SELANGOR'), '43800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEPANG')&(mukim_district_state['state']=='SELANGOR'), '43900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEPANG')&(mukim_district_state['state']=='SELANGOR'), '43950', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BARU BANGI')&(mukim_district_state['state']=='SELANGOR'), '43650', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BARU SALAK TINGGI')&(mukim_district_state['state']=='SELANGOR'), '43900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LAPANGAN TERBANG ANTARABANGSA SEPANG')&(mukim_district_state['state']=='SELANGOR'), '43900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI MERAB')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DENGKIL')&(mukim_district_state['state']=='SELANGOR'), '43800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SALAK')&(mukim_district_state['state']=='SELANGOR'), '43900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI PELEK')&(mukim_district_state['state']=='SELANGOR'), '43950', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU 1 SEPANG')&(mukim_district_state['state']=='SELANGOR'), '43900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT BISA')&(mukim_district_state['state']=='SELANGOR'), '62000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT PRANG')&(mukim_district_state['state']=='SELANGOR'), '43800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DATO BAKAR BAGINDA')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BARU SALAK TINGGI')&(mukim_district_state['state']=='SELANGOR'), '43900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI MERAB')&(mukim_district_state['state']=='SELANGOR'), '43000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJUNG MAS')&(mukim_district_state['state']=='SELANGOR'), '43900', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SELANGOR')]
mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\2555379758.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KLANG')&(mukim_district_state['state']=='SELANGOR'), '41400', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\2555379758.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postco

postcode      165
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [43]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='WP KUALA LUMPUR')]

mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '42800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHERAS')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PETALING')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '46200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PETALING JAYA')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '46200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU KELANG')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SETAPAK')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '53100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '52100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHERAS')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '56000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PETALING')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '46675', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SETAPAK')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '53300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA LUMPUR')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '50450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PETALING JAYA')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '46200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU KELANG')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA LUMPUR')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '50450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '51200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AMPANG')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI PENCHALA')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AMPANG')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANDAR BAHARU SUNGAI BESI')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '57000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '51200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU CAVES')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '68100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KEPONG')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '52100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA PAUH')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '50460', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PETALING')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '50603', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SALAK SOUTH')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '57100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI PENCHALA')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PETALING')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '50603', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='WP KUALA LUMPUR')]
mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\843836476.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '42800', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\843836476.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['po

postcode      151
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [44]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SARAWAK')]

# kuching
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUEH LAND')&(mukim_district_state['district']=='KUCHING'), '94100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='STUNGKOR')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SAMPADI')&(mukim_district_state['district']=='KUCHING'), '93070', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JAGOI')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SENGGI POAK')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SALAK')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PANGKALAN AMPAT')&(mukim_district_state['district']=='KUCHING'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUCHING UTARA')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUCHING TENGAH')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUCHING')&(mukim_district_state['district']=='KUCHING'), '93100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SENTAH-SEGU')&(mukim_district_state['district']=='KUCHING'), '94200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU KAWA')&(mukim_district_state['district']=='KUCHING'), '93250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI TENGAH')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI KITANG')&(mukim_district_state['district']=='KUCHING'), '93250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU KITANG')&(mukim_district_state['district']=='KUCHING'), '93250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU 15, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='175, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JALAN BATU 19, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU 24, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PANGKALAN KUT')&(mukim_district_state['district']=='KUCHING'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BELIONG')&(mukim_district_state['district']=='KUCHING'), '94300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BAKO')&(mukim_district_state['district']=='KUCHING'), '93010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JANGKAR')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR RAMBUNGAN')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR STUNKOR')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KRANJI')&(mukim_district_state['district']=='KUCHING'), '94750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SINIWAN')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MUSI')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAI TON')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BIDI')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PANGKALAN TEBANG')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PEJIRU')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BARATOK')&(mukim_district_state['district']=='KUCHING'), '94200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TERBAT')&(mukim_district_state['district']=='KUCHING'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JALAN BATU 17')&(mukim_district_state['district']=='KUCHING'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JALAN BATU 17, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GADING LUNDU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MATANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUARA TEBAS')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '93010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU 8, JALAN MATANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEMATANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LUNDU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PAKU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JAMBUSAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BUSO')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TUNDONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KROKONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TIANG BEKAP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '93010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAPAH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SIBURAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KUCHING'), '94200', mukim_district_state['postcode'])


# sri aman
''' 
postcode	location	district	state	mukim
58043	NaN	NaN	SRI AMAN	SARAWAK	UNDUP
58044	NaN	NaN	SRI AMAN	SARAWAK	KLAUH
58045	NaN	NaN	SRI AMAN	SARAWAK	BIJAT
58046	NaN	NaN	SRI AMAN	SARAWAK	SKARANG
58047	NaN	NaN	SRI AMAN	SARAWAK	KERANGGAS
58048	NaN	NaN	SRI AMAN	SARAWAK	MARUP
58049	NaN	NaN	SRI AMAN	SARAWAK	LAMANAK
58050	NaN	NaN	SRI AMAN	SARAWAK	BUKIT BESAI
58051	NaN	NaN	SRI AMAN	SARAWAK	AI ENGKARI
58052	NaN	NaN	SRI AMAN	SARAWAK	LESONG
58053	NaN	NaN	SRI AMAN	SARAWAK	SELANJANG
58054	NaN	NaN	SRI AMAN	SARAWAK	SILANTEK
58055	NaN	NaN	SRI AMAN	SARAWAK	SIRNANGGANG
58056	NaN	NaN	SRI AMAN	SARAWAK	BANDAR LINGGA
58057	NaN	NaN	SRI AMAN	SARAWAK	BANDAR LUBOK ANTU
58058	NaN	NaN	SRI AMAN	SARAWAK	BANDAR ENGKILILI
58059	NaN	NaN	SRI AMAN	SARAWAK	BANDAR BATU LINTANG
58060	NaN	NaN	SRI AMAN	SARAWAK	BANDAR BANTING
58061	NaN	NaN	SRI AMAN	SARAWAK	BANDAR PANTU
58062	NaN	NaN	SRI AMAN	SARAWAK	BANDAR BAKONG
58063	NaN	NaN	SRI AMAN	SARAWAK	BANDAR UNDUP
58064	NaN	NaN	SRI AMAN	SARAWAK	BANDAR SKRANG
58065	NaN	NaN	SRI AMAN	SARAWAK	BANDAR MELUGU
58066	NaN	NaN	SRI AMAN	SARAWAK	BANDAR SABU

mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='UNDUP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KLAUH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BIJAT')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SKARANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERANGGAS')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95007', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MARUP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LAMANAK')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT BESAI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AI ENGKARI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LESONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELANJANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SILANTEK')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIRNANGGANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LINGGA')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LUBOK ANTU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ENGKILILI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU LINTANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANTING')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PANTU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BAKONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR UNDUP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SKRANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MELUGU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SABU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SRI AMAN'), '', mukim_district_state['postcode'])

'''
#sibu
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEDUAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ENGKILO')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASAI-SIONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ASSAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MENYAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KABANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LUKUT')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MAPAI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MAROH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SPALI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='QYA-DALAT')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SPAPA')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PAKU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LALAI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKAH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GIGIS')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELANGAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BALINGAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ARIP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PELUNGAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAWAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BULOH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SIBU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SENGEI MERAH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TEKU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR DURIN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KANOWIT')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR DAP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MACHAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR NGEMAH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SENGAYAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SIBINTEK')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU'), '', mukim_district_state['postcode'])

'''

#miri
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KONSESI MIRI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LUTONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BAZAR JALAN RIAM')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA BARAM')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LAMBIR')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA NYABOR')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAREO')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='APOH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LIO MATOH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SILAT')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TUTOH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PATAH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LEPU LEJU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='MIRI'), '', mukim_district_state['postcode'])

'''


#limbang
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DANAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PANARUAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TRUSAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LAWAS')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MERAPOK')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LIMBANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NANGA MEDAMIT')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='UKONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNDAR')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI ADANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LONG NAPIR')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI ADDANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TENGOA-SUKANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LONG NERAPAP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LONG SEMADO')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAKALALAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU LAWI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='LIMBANG'), '', mukim_district_state['postcode'])

'''


#sarikei
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERENDANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MARADONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TULAI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SARIKEI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SARE')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEDANUM')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MELURUN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JIKANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BINATANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JULAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PAKAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SELALANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GUNUNG AYER')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BINATANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SARIKEI'), '', mukim_district_state['postcode'])
'''

#kapit
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KATIBAS')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='IBAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MENUAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='OYAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANING')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MAJAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MENRAL')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='METAH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RIRONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MAMU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ANGKUAH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PELAGUS')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KAPIT')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANGKIT')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU LAGA')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PELANDUK')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ENTEMU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MENGIONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERANI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BALUI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUMBONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MURUM')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUNAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DANUM')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='KAPIT'), '', mukim_district_state['postcode'])

'''


#betong
''' 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MALUDAM')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BETONG'), '', mukim_district_state['postcode'])
'''


# bintulu
''' 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BINTULU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEMENA')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEBAUH')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LANANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PANDAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TUBAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELEZU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU KAPAL')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RASAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PANDAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA TATAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TATAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUAN LAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SANGAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ANAP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAKUS')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JELALONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BINI')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='BINTULU'), '', mukim_district_state['postcode'])

'''

# samarahan
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MENUKU')&(mukim_district_state['district']=='SAMARAHAN'), '94300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SAMARAHAN')&(mukim_district_state['district']=='SAMARAHAN'), '94300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MELIKIN LAND')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUNDA-SABAL')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI MERAH')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNAGAI MERANG')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI PALAH')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TEDEBU')&(mukim_district_state['district']=='SAMARAHAN'), '94760', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BALAI RINGIN')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAMBIRAT')&(mukim_district_state['district']=='SAMARAHAN'), '94600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SIMUJAN')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TEBELU')&(mukim_district_state['district']=='SAMARAHAN'), '94850', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TERBAT')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])

'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LESONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAYAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUARA TUANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKAR-SADONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI KEDUP')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEDILU-GEDONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SADONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEBANGAN-KEPAYAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEBUYAU')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU 29, JALAN SIMANGGANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU 32, JALAN SIMANGGANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU 34+, JALAN SIMANGGANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SERIAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TEBAKANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MUARA MONGKOS')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MUARA TUANG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GEDONG')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEBANGAN')&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SAMARAHAN'), '', mukim_district_state['postcode'])

'''
mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SARAWAK')]
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SARAWAK')&(mukim_district_state['district']=='SIBU')]

mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\893868606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUEH LAND')&(mukim_district_state['district']=='KUCHING'), '94100', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\893868606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['po

postcode      104
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [45]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='KELANTAN')]

mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ENGGONG')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GAJAH')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KANDEK')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KENOR')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA GERIS')&(mukim_district_state['district']=='KUALA KRAI'), '18200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KRAI')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA NAL')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA PERGAU')&(mukim_district_state['district']=='KUALA KRAI'), '18200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA STONG')&(mukim_district_state['district']=='KUALA KRAI'), '18200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MAMBONG')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MANJOR')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELEKONG')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KRAI')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='KELANTAN')]
mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\3504162356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ENGGONG')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\3504162356.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state[

postcode       91
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [46]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='MELAKA')]
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ALAI')&(mukim_district_state['district']=='MELAKA TENGAH'), '75460', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BACHANG')&(mukim_district_state['district']=='MELAKA TENGAH'), '75350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERTAM')&(mukim_district_state['district']=='MELAKA TENGAH'), '76450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHENG')&(mukim_district_state['district']=='MELAKA TENGAH'), '75250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KRUBONG')&(mukim_district_state['district']=='MELAKA TENGAH'), '75260', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG TEMU')&(mukim_district_state['district']=='MELAKA TENGAH'), '75050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PRINGGIT')&(mukim_district_state['district']=='MELAKA TENGAH'), '75400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANGGA BATU')&(mukim_district_state['district']=='MELAKA TENGAH'), '76400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG KELING')&(mukim_district_state['district']=='MELAKA TENGAH'), '76400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG MINYAK')&(mukim_district_state['district']=='MELAKA TENGAH'), '75260', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOK MAS')&(mukim_district_state['district']=='MELAKA TENGAH'), '75460', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '75400', mukim_district_state['postcode'])

#melaka tengah
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER MOLEK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BALAI PANJANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU BERENDAM')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT BARU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT KATIL')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT LINTANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT PIATU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT RAMBAI')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DUYONG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='UJONG PASIR')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KANDANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KLEBANG BESAR')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KLEBANG KECHIL')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG SEMABOK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PAYA RUMPUT')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERNU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMABOK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI UDANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BUKIT BARU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AYER MOLEK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU BERENDAM')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT RAMBAI')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KANDANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KLEBANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PAYA RUMPUT')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI UDANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANGGA BATU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJONG KLING')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '', mukim_district_state['postcode'])

'''
#jasin
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER PANAS')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG MALAKA')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT SENGGEH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHABAU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHIN CHIN')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHOHONG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JASIN')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JUS')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KESANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MERLIMAU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NYALAS')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RIM')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEBATU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELANDAR')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMPANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMUJOK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERKAM')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGEI RAMBAI')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEDONG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='UMBAI')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JASIN')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MERLIMAU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN ASAHAN')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATANG MALAKA')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BEMBAN')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHIN CHIN')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KESANG PAJAK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN NYALAS')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SELANDAR')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SEMPANG BEKOH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI RAMBAI')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='JASIN'), '', mukim_district_state['postcode'])

'''
#alor gajah
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER PA\'ABAS')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELIMBING')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERINGIN')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BRISU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DURIAN TUNGGAL')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GADEK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KELEMAK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEMUNING')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA LINGGI')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA SUNGEI BARU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LENDU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MACHAP')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MASJID TANAH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MELAKA PINDAH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MELEKEK')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG SEBANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PARIT MELANA')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEGOH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU SEBANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RAMUAN CHINA BESAR')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RAMUAN CHINA KECHIL')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='REMBIA')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGEI BARU HILIR')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGEI BARU TENGAH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGEI BARU ULU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGEI BULOH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGEI PETAI')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGEI SIPUT')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TABOH NANING')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG RIMAU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEBONG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ALOR GAJAH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MASJID TANAH')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PULAU SEBANG')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DURIAN TUNGGAL')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA SUNGAI BARU')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LUBOK CHINA')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN REMBIA')&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='ALOR GAJAH'), '', mukim_district_state['postcode'])

'''



mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH')]

mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\3189086293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ALAI')&(mukim_district_state['district']=='MELAKA TENGAH'), '75460', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\3189086293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state[

postcode       79
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [47]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='NEGERI SEMBILAN')]

#jelebu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELAMI LEMI')&(mukim_district_state['district']=='JELEBU'), '71600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KENABOI')&(mukim_district_state['district']=='JELEBU'), '71650', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KELAWANG')&(mukim_district_state['district']=='JELEBU'), '71600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TERIANG HILIR')&(mukim_district_state['district']=='JELEBU'), '72400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KELAWANG')&(mukim_district_state['district']=='JELEBU'), '71600', mukim_district_state['postcode'])

'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERADONG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JELEBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERTANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JELEBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA KELAWANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JELEBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PERTANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JELEBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TITI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JELEBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG DURIAN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JELEBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIMPANG PERTANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JELEBU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI MUNTOH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JELEBU'), '', mukim_district_state['postcode'])
'''

#kuala pilah
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AMPANG TINGGI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU JEMPOL')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU MUAR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JOHOL')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JUASSEH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEPIS')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LANGKAP')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PARIT TINGGI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PILAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SRI MENANTI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TERACHI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA PILAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DANGI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GUNUNG PASIR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JOHOL')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PARIT TINGGI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SENALING')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANJONG IPOH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JUASSEH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT GELUGOR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MELANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AR MAWANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DANGI BARU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='KUALA PILAH'), '', mukim_district_state['postcode'])

''' 

#port dickson
''' 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JIMAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LINGGI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PORT DICKSON')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SI RUSA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PORT DICKSON')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TELUK KEMANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LUKUT')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PASIR PANJANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PENGKALAN KEMPAS')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHUAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PORT DICKSON')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TELUK KEMANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT PELANDUK')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LINGGI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AIR KUNING')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI MENYALA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BAGAN PINANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANAH MERAH UTARA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TANAH MERAH SELATAN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JEMIMA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='PORT DICKSON'), '', mukim_district_state['postcode'])
'''

#rembau
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU HAMPAR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BONGEK')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHEMBONG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHENGKAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GADONG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUNDOR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LEGONG ILIR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LEGONG ULU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MIKU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NERASAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEDAS')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PILIN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELEMAK')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMERBOK')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SPRI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG KELING')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TITIAN BINTANGOR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR REMBAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPONG BATU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KOTA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LUBOK CHINA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PEDAS')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHEMBONG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN REMBAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHENGKAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SERI KOTA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SERI KENDONG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MERBAU SEMBILAN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='REMBAU'), '', mukim_district_state['postcode'])

''' 

#seremban
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LABU')&(mukim_district_state['district']=='SEREMBAN'), '71900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEREMBAN')&(mukim_district_state['district']=='SEREMBAN'), '70000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SETUL')&(mukim_district_state['district']=='SEREMBAN'), '71800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEREMBAN')&(mukim_district_state['district']=='SEREMBAN'), '70000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEREMBAN UTAMA')&(mukim_district_state['district']=='SEREMBAN'), '70000', mukim_district_state['postcode'])
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AMPANGAN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LENGGENG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PANTAI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RANTAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RASAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEREMBAN 3')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MANTIN UTAMA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BARU ENSTEK')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BARU KOTA SRI MAS')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR NILAI UTAMA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SRI SENDAYAN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BROGA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN ULU BERANANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LABU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LENGGENG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MAMBAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN NILAI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MANTIN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PAJAM')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN RANTAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TIROI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PANCOR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TAMAN SEREMBAN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN RAHANG BARU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PAROI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT KEPAYANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DUSUN SETIA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SUNGAI GADUT')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKTI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SIKAMAT')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SETUL')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SHAH BANDAR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN ULU TEMIANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PAROI JAYA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN RASAH JAYA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SENAWANG')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SEREMBAN JAYA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='SEREMBAN'), '', mukim_district_state['postcode'])
''' 

#tampin
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER KUNING')&(mukim_district_state['district']=='TAMPIN'), '73200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GEMAS')&(mukim_district_state['district']=='TAMPIN'), '73400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERU')&(mukim_district_state['district']=='TAMPIN'), '73000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='REPAH')&(mukim_district_state['district']=='TAMPIN'), '73000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEBONG')&(mukim_district_state['district']=='TAMPIN'), '73000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GEMAS')&(mukim_district_state['district']=='TAMPIN'), '73400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAMPIN')&(mukim_district_state['district']=='TAMPIN'), '73000', mukim_district_state['postcode'])
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GEMENCHEH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TAMPIN TENGAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AIR KUNING SELATAN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATANG MELAKA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GEMENCHECH BAHRU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN REPAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TAMPIN TENGAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AIR KUNING')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PASIR BESAR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN REPAH PERMAI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN REPAH JAYA')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='TAMPIN'), '', mukim_district_state['postcode'])

''' 


#jempol
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JELAI')&(mukim_district_state['district']=='JEMPOL'), '73500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ROMPIN')&(mukim_district_state['district']=='JEMPOL'), '73430', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SERTING')&(mukim_district_state['district']=='JEMPOL'), '72100', mukim_district_state['postcode'])
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA JEMPOL')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERTING HILIR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERTING ULU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BAHAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SERI JEMPOL')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BAHAU')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BATU KIKIR')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA JELAI')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN ROMPIN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN LADANG GEDDES')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MAHSAN')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SERTING TENGAH')&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL'), '', mukim_district_state['postcode'])

''' 



mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL')]

mukim_district_state.isnull().sum()


C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1312657637.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELAMI LEMI')&(mukim_district_state['district']=='JELEBU'), '71600', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1312657637.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state[

postcode       59
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [48]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='JOHOR')]

# batu pahat
''' 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAGAN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHAAH BAHRU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAMPONG BAHRU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LINAU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LUBOK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MINYAK BEKU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PESERAI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIMPANG KANAN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SIMPANG KIRI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SRI GADING')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SRI MEDAN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI KLUANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI PUNGGOR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG SEMBRONG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR AYER HTAM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANDAR PENGGARAM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR RENGIT')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SENGGARANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR YONG PENG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='BATU PAHAT'), '', mukim_district_state['postcode'])

'''
# johor bahru
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PLENTONG')&(mukim_district_state['district']=='JOHOR BAHRU'), '81750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAI')&(mukim_district_state['district']=='JOHOR BAHRU'), '81300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMBRAU')&(mukim_district_state['district']=='JOHOR BAHRU'), '81100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JOHOR BAHRU')&(mukim_district_state['district']=='JOHOR BAHRU'), '80000', mukim_district_state['postcode'])
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JELUTONG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='JOHOR BAHRU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI TIRAM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='JOHOR BAHRU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJUNG KUPANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='JOHOR BAHRU'), '', mukim_district_state['postcode'])
'''

# kluang
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU BENUT')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAHANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KLUANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LAYANG-LAYANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MACHAP')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NIYOR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PALOH')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RENGAM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KLUANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PALOH')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR RENGAM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KLUANG'), '', mukim_district_state['postcode'])

'''

# kota tinggi 
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU SUNGAI JOHOR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU SUNGAI SELIDI BESAR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JOHOR LAMA')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAMBAU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KOTA TINGGI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PANTAI TIMUR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGGERANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELIDI BESAR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELIDI KECHIL')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJUNG SURAT')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KOTA TINGGI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KOTA TINGGI'), '', mukim_district_state['postcode'])

'''

# mersing
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JEMALUANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LENGGOR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MERSING')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG ENDAU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENYABONG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU AUR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU BABI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU PEMANGGIL')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU SIBU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU TINGGI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMBRONG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TENGGAROH')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TENGLU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TRIANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JEMALUANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MERSING')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MERSING KANAN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PADANG ENDAU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MERSING'), '', mukim_district_state['postcode'])

'''

# muar
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER HITAM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT KEPONG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JALAN BAKRI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JORAK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LENGA')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PARIT BAKAR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PARIT JAWA')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SRI MERANTI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI BALANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI RAYA & KAMPUNG BUKIT PASIR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI TERAP')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANDAR MAHARANI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BUKIT KEPONG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PANCHOR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PARIT JAWA')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT PASIR')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='MUAR'), '', mukim_district_state['postcode'])

'''

#pontian
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER BALOI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AIR MASIN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='API-API')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BENUT')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JERAM BATU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGKALAN RAJA')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PONTIAN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RIMBA TERJUN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERKAT')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI KARANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGEI PINGGAN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BENUT')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PONTIAN KECHIL')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN PEKAN NENAS')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='PONTIAN'), '', mukim_district_state['postcode'])
'''

# segamat 
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BEKOK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BULOH KASAP')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHAAH')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GEMAS')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GEMEREH')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JABI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JEMENTAH')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LABIS')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='POGOH')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERMIN')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI SEGAMAT')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU ANAM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BEKOK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BULOH KASAP')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JEMENTAH')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BADNAR LABIS')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEGAMAT')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GEMAS BAHRU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='SEGAMAT'), '', mukim_district_state['postcode'])
'''

# kulaijaya 
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KULAI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KULAIJAYA'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SENAI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KULAIJAYA'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEDENAK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KULAIJAYA'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT BATU')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KULAIJAYA'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KULAI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='KULAIJAYA'), '', mukim_district_state['postcode'])
'''

# ledang 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUNDANG')&(mukim_district_state['district']=='LEDANG'), '84600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BADNAR BUKIT KANGKAR')&(mukim_district_state['district']=='LEDANG'), '84300', mukim_district_state['postcode'])
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANGKAK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT SERAMPANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GERSIK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEROM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KESANG')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PARIT BUNGA')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEROM')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI MATI')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANGKAK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN GRISEK')&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG'), '', mukim_district_state['postcode'])
'''

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG')]

mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1508872861.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PLENTONG')&(mukim_district_state['district']=='JOHOR BAHRU'), '81750', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1508872861.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_st

postcode       53
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [49]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='TERENGGANU')]

#kuala terengganu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDI')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANGGUL')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU JABUR')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24060', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR SEMUT')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEBAK')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELUK KALUNG')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU BURUK')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU RAKIT')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21020', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CABANG TIGA')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CENERIANG')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21080', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELUGUR KEDAI')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELUGUR RAJA')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEPUNG')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA IBAI')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA NERUS')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PALUH')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGADANG BULUH')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21070', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU-PULAU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RENGAS')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERADA')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TOK JAMAL')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20000', mukim_district_state['postcode'])
''' 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ATAS TOL')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELARA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT BESAR')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUBANG PARIT')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LOSONG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MANIR')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU REDANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CABANG TIGA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '', mukim_district_state['postcode'])

''' 

#hulu terengganu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU BERANG')&(mukim_district_state['district']=='HULU TERENGGANU'), '21800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU TELEMUNG')&(mukim_district_state['district']=='HULU TERENGGANU'), '21700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU TERENGGANU')&(mukim_district_state['district']=='HULU TERENGGANU'), '21700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JENAGUR')&(mukim_district_state['district']=='HULU TERENGGANU'), '21700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA TELEMUNG')&(mukim_district_state['district']=='HULU TERENGGANU'), '21210', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGHULU DIMAN')&(mukim_district_state['district']=='HULU TERENGGANU'), '21010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANGGUL')&(mukim_district_state['district']=='HULU TERENGGANU'), '21040', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TERSAT')&(mukim_district_state['district']=='HULU TERENGGANU'), '21700', mukim_district_state['postcode'])
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA BERANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='HULU TERENGGANU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA BERANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='HULU TERENGGANU'), '', mukim_district_state['postcode'])
'''


#besut
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT KENAK')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT PUTERI')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU BESUT')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JABI')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAMPONG RAJA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KELUANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERANDANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA BESUT')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUBANG BEMBAN')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LUBUK KAWAH')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR AKAR')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PELAGAT')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGKALAN NANGKA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU PERHENTIAN')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMBILA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TENANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN JERTIH')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KAMPUNG RAJA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA BESUT')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='BESUT'), '', mukim_district_state['postcode'])

'''
#dungun
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA ABANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BESUL')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU PAKA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JENGAI')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JERANGAU')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA DUNGUN')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA PAKA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUMPAL')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR RAJA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RASAU')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SURA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR DUNGUN')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA PAKA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='DUNGUN'), '', mukim_district_state['postcode'])

'''

#marang
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JERUNG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='MARANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MERCANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='MARANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU KERENGGA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='MARANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RUSILA')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='MARANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ALUR LIMBAT')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='MARANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT PAYUNG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='MARANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MARANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='MARANG'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN BUKIT PAYUNG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='MARANG'), '', mukim_district_state['postcode'])

'''
#setiu
'''
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CALUK')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='SETIU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GUNTUNG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='SETIU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU NERUS')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='SETIU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MERANG')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='SETIU'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TASIK')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='SETIU'), '', mukim_district_state['postcode'])
'''

#kemaman
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDI')&(mukim_district_state['district']=='KEMAMAN'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANGGUL')&(mukim_district_state['district']=='KEMAMAN'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU JABUR')&(mukim_district_state['district']=='KEMAMAN'), '24060', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR SEMUT')&(mukim_district_state['district']=='KEMAMAN'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEBAK')&(mukim_district_state['district']=='KEMAMAN'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELUK KALUNG')&(mukim_district_state['district']=='KEMAMAN'), '24100', mukim_district_state['postcode'])
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BINJAI')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CUKAI')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU CUKAI')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEMASIK')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERTEH')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KIJAL')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR CUKAI')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AIR JERNIH')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN AIR PUTIH')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KEMASIK')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KIJAL')&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN'), '', mukim_district_state['postcode'])

'''

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN')]

mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\2378597359.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDI')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\2378597359.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_st

postcode       23
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [50]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PAHANG')]

#bentong
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BENTONG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BENTONG'), '28700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='SABAI')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BENTONG'), '28700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PELANGAI')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BENTONG'), '28700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BANDAR BENTONG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BENTONG'), '28700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BANDAR KARAK')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BENTONG'), '28600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN TELEMONG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BENTONG'), '28500', mukim_district_state['postcode'])


'''

#cameron highlands
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU TELOM')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39000', mukim_district_state['postcode'])
''' 
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='RINGLET')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='TANAH RATA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BANDAR TANAH RATA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN BERINCHANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN LUBOK TAMANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN RINGLET')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39200', mukim_district_state['postcode'])

'''


#jerantut
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BURAU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='ULU CHEKA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='ULU TEMBELING')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='KELOLA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='KUALA TEMBELING')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEDAH')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PULAU TAWAR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='TEBING TINGGI')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='TEH')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='TEMBELING')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BANDAR JERANTUT')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN KUALA TEMBELING')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN JERANSANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='JERANTUT'), '27000', mukim_district_state['postcode'])

'''

#kuantan### 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU LEPAR')&(mukim_district_state['district']=='KUANTAN'), '26330', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU KUANTAN')&(mukim_district_state['district']=='KUANTAN'), '26200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KUANTAN')&(mukim_district_state['district']=='KUANTAN'), '25100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENOR')&(mukim_district_state['district']=='KUANTAN'), '26060', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUANTAN')&(mukim_district_state['district']=='KUANTAN'), '25000', mukim_district_state['postcode'])
'''
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BESERAH')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='KUANTAN'), '26100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='SUNGAI KARANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='KUANTAN'), '26100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BANDAR GAMBANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='KUANTAN'), '26300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN BESERAH')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='KUANTAN'), '26100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN TANJNG LUMPUR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='KUANTAN'), '26060', mukim_district_state['postcode'])
'''

#lipis
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU YON')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUDU')&(mukim_district_state['district']=='LIPIS'), '27310', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHEKA')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GUA')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU JELAI')&(mukim_district_state['district']=='LIPIS'), '27650', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KECHAU')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELANG')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='KUALA LIPIS')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PENJOM')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27210', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='TANJUNG BESAR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27220', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BANDAR KUALA LIPIS')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BENTA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PADANG TENGKU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN PADANG TENGKU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN TAMAN JELAI')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN PENJOM')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27210', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN MELA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN KERAMBIT')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN RPSB KG. PAGAR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN MERAPUH')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN KECHAU TUI')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])

'''


#pekan
'''
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BEBAR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='GANCHONG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='KUALA PAHANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='LEPAR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PAHANG TUA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PENYOR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PULAU MANIS')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PULAU RUSA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='TEMAI')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='BANDAR PEKAN')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN KUALA PAHANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['mukim']=='PEKAN NENASI')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='PEKAN'), '26700', mukim_district_state['postcode'])

'''

#raub
'''   
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU TALAM')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DONG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GALI')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU DONG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEGA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMANTAN ULU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TERAS')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR RAUB')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TERAS')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHEROH')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DONG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TRANUM')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='RAUB'), '', mukim_district_state['postcode'])

'''

#temerloh
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANGAU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JENDERAK')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERDAU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LEBAK')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LIPAT KIJANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MENTAKAB')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERAK')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SANGGANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMANTAN')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SONGSANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MENTAKAB')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TEMERLOH')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KERDAU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA KERAU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LANCHANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='TEMERLOH'), '', mukim_district_state['postcode'])

'''

#rompin
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERATONG')&(mukim_district_state['district']=='ROMPIN'), '26900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ROMPIN')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TIOMAN')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ROMPIN I')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ROMPIN II')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ROMPIN III')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ROMPIN IV')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TIOMAN')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ENDAU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='ROMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PONTIAN')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='ROMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PONTIAN')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='ROMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ENDAU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='ROMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MUADZAM SHAH I')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='ROMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MUADZAM SHAH II')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='ROMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN KUALA ROMPIN')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='ROMPIN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN TIOMAN')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='ROMPIN'), '', mukim_district_state['postcode'])

'''

#maran
''' 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT SEGUMPAL')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='MARAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHENOR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='MARAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERTAU')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='MARAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LUIT')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='MARAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MARAN')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='MARAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN CHENOR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='MARAN'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN SRI JAYA')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='MARAN'), '', mukim_district_state['postcode'])

'''

#bera
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERA')&(mukim_district_state['district']=='BERA'), '28200', mukim_district_state['postcode'])
'''  
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TRIANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BERA'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TRIANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BERA'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN DURIAN TAWAR')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BERA'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MENGKARAK')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BERA'), '', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKAN MENGKUANG')&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BERA'), '', mukim_district_state['postcode'])

'''

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BENTONG')]
mukim_district_state.isnull().sum()


C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1458136286.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU TELOM')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39000', mukim_district_state['postcode'])
C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1458136286.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_dis

postcode        1
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

In [51]:
mukim_district_state[mukim_district_state['postcode'].isnull()]

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3
57545,NaN,NaN,W. P. PUTRAJAYA,WP PUTRAJAYA,BANDAR PUTRAJAYA,NaN,NaN,NaN


In [52]:
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PUTRAJAYA')&(mukim_district_state['district']=='W. P. PUTRAJAYA'), '62100', mukim_district_state['postcode'])

mukim_district_state.isnull().sum()

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\1672821226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PUTRAJAYA')&(mukim_district_state['district']=='W. P. PUTRAJAYA'), '62100', mukim_district_state['postcode'])


postcode        0
location     1970
district       19
state           0
mukim           0
postcode1     790
postcode2     625
postcode3     470
dtype: int64

- Check if No Null Value for Postcode

In [53]:
mukim_district_state[(mukim_district_state['postcode'].isnull())]

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3


In [56]:
mukim_district_state

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3
56234,01000,NaN,PERLIS,PERLIS,ABI,NaN,NaN,NaN
56235,02600,NaN,PERLIS,PERLIS,ARAU,02600,02600,02600
56236,02400,NaN,PERLIS,PERLIS,BERSERI,NaN,NaN,NaN
56237,02500,NaN,PERLIS,PERLIS,CHUPING,NaN,NaN,NaN
56238,01000,NaN,PERLIS,PERLIS,UTAN AJI,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
58219,97000,NaN,BINTULU,SARAWAK,ANAP,97000,97000,97000
58220,97000,NaN,BINTULU,SARAWAK,KAKUS,97000,"97000, 97200",97000
58221,97000,NaN,BINTULU,SARAWAK,JELALONG,97000,97000,97000
58222,97000,NaN,BINTULU,SARAWAK,BINIA,97000,97000,97000


In [68]:
postcode_group_location=location.groupby('postcode')['location'].apply(list).reset_index()
postcode_group_location['location']=postcode_group_location['location'].apply(lambda x: ', '.join(x))
postcode_group_location

,postcode,location
0,01000,"ALOR REDEH, ALOR TOK TERANG, BOHOR GELAM, JALA..."
1,01007,PETI SURAT
2,01009,BEG BERKUNCI
3,01500,DYMM TUANKU RAJA PERLIS
4,01502,YAB MENTERI BESAR PERLIS
...,...,...
2773,98750,NANGA MEDAMIT
2774,98800,"KAMPUNG ARU, KAMPUNG AWAT-AWAT, KAMPUNG BARU, ..."
2775,98850,"AWAT-AWAT, BA KELALAN, BELAKANG KEDAI, KAMPUNG..."
2776,98857,PETI SURAT


In [72]:
# fill na location mukim_district_state with postcode_group_location
mukim_district_state['location']=mukim_district_state['postcode'].map(postcode_group_location.set_index('postcode')['location'])
mukim_district_state = mukim_district_state.drop(columns=['postcode1', 'postcode2', 'postcode3']).reset_index(drop=True)
mukim_district_state

C:\Users\izard\AppData\Local\Temp\ipykernel_19484\230210260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mukim_district_state['location']=mukim_district_state['postcode'].map(postcode_group_location.set_index('postcode')['location'])


,postcode,location,district,state,mukim
0,01000,"ALOR REDEH, ALOR TOK TERANG, BOHOR GELAM, JALA...",PERLIS,PERLIS,ABI
1,02600,"ARAU, GUAR SANJI, KAMPUNG ALOR MERAH, KAMPUNG ...",PERLIS,PERLIS,ARAU
2,02400,"BESERI DALAM, BESERI, BILAL UDUH, BOHOR MADI, ...",PERLIS,PERLIS,BERSERI
3,02500,"BALAI POLIS MATA AIR, BUKIT CHUPING, FELDA CHU...",PERLIS,PERLIS,CHUPING
4,01000,"ALOR REDEH, ALOR TOK TERANG, BOHOR GELAM, JALA...",PERLIS,PERLIS,UTAN AJI
...,...,...,...,...,...
1965,97000,"ABANG GALAU, BINTULU, KAMPUNG BARU, KAMPUNG DA...",BINTULU,SARAWAK,ANAP
1966,97000,"ABANG GALAU, BINTULU, KAMPUNG BARU, KAMPUNG DA...",BINTULU,SARAWAK,KAKUS
1967,97000,"ABANG GALAU, BINTULU, KAMPUNG BARU, KAMPUNG DA...",BINTULU,SARAWAK,JELALONG
1968,97000,"ABANG GALAU, BINTULU, KAMPUNG BARU, KAMPUNG DA...",BINTULU,SARAWAK,BINIA


In [74]:
mukim_district_state.to_csv('../output/malaysia-postcodes-location-mukim-district-state.csv', index=False)